In [303]:
# pip install --upgrade kiwipiepy

In [1]:
# pip install kiwipiepy==0.18.1

  Using cached kiwipiepy-0.18.1-cp311-cp311-win_amd64.whl.metadata (1.1 kB)
  Using cached kiwipiepy_model-0.18.0-py3-none-any.whl
Using cached kiwipiepy-0.18.1-cp311-cp311-win_amd64.whl (1.3 MB)
  Attempting uninstall: kiwipiepy-model
    Found existing installation: kiwipiepy_model 0.19.0
    Uninstalling kiwipiepy_model-0.19.0:
      Successfully uninstalled kiwipiepy_model-0.19.0
  Attempting uninstall: kiwipiepy
    Found existing installation: kiwipiepy 0.19.0
    Uninstalling kiwipiepy-0.19.0:
      Successfully uninstalled kiwipiepy-0.19.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
import openai # 0.28.0
import re
from kiwipiepy import Kiwi

# Kiwi 초기화
kiwi = Kiwi(num_workers=0, model_path=None, load_default_dict=True, integrate_allomorph=True, model_type='sbg', typos=None, typo_cost_threshold=2.5)

"""
num_workers: 2 이상이면 단어 추출 및 형태소 분석에 멀티 코어를 활용하여 조금 더 빠른 속도로 분석을 진행할 수 있습니다.
1인 경우 단일 코어만 활용합니다. num_workers가 0이면 현재 환경에서 사용가능한 모든 코어를 활용합니다.
생략 시 기본값은 0입니다.
model_path: 형태소 분석 모델이 있는 경로를 지정합니다. 생략시 kiwipiepy_model 패키지로부터 모델 경로를 불러옵니다.
load_default_dict: 추가 사전을 로드합니다. 추가 사전은 위키백과의 표제어 타이틀로 구성되어 있습니다. 이 경우 로딩 및 분석 시간이 약간 증가하지만 다양한 고유명사를 좀 더 잘 잡아낼 수 있습니다. 분석 결과에 원치 않는 고유명사가 잡히는 것을 방지하려면 이를 False로 설정하십시오.
integrate_allomorph: 어미 중, '아/어', '았/었'과 같이 동일하지만 음운 환경에 따라 형태가 달라지는 이형태들을 자동으로 통합합니다.
model_type: 형태소 분석에 사용할 언어 모델을 지정합니다. 'knlm', "sbg" 중 하나를 선택할 수 있습니다. "sbg" 는 상대적으로 느리지만 먼 거리에 있는 형태소 간의 관계를 포착할 수 있습니다.
typos: 형태소 분석 시 간단한 오타를 교정합니다. None으로 설정 시 교정을 수행하지 않습니다.
typo_cost_threshold: 오타 교정을 허용할 최대 오타 비용을 설정합니다.
"""

# API 키 설정
openai.api_key = "sk-proj-nlhN73CnCzO3ShLYyCPuT3BlbkFJdzOuNYCbeHCwAhrhhh7p"

        
######## 데이터 => src/v1/resoures/ 디렉토리 내부로 이동시켜서 불러오게 바꾸기
pos_descriptions  = {"NNG"	:"일반 명사",
"NNP"	:"고유 명사",
"NNB"	:"의존 명사",
"NR"	:"수사",
"NP"	:"대명사",
"VV"	:"동사",
"VV-R"	:"동사(규칙)",
"VV-I"	:"동사(불규칙)",
"VA"	:"형용사",
"VA-R"	:"형용사(규칙)",
"VA-I"	:"형용사(불규칙)",
"VX"	:"보조 용언",
"VX-R"	:"보조 용언(규칙)",
"VX-I"	:"보조 용언(불규칙)",
"VCP"	:"긍정 지시사(이다)",
"VCN"	:"부정 지시사(아니다)",
"MM"	:"관형사",
"MAG"	:"일반 부사",
"MAJ"	:"접속 부사",
"IC"	:"감탄사",
"JKS"	:"주격 조사",
"JKC"	:"보격 조사",
"JKG"	:"관형격 조사",
"JKO"	:"목적격 조사",
"JKB"	:"부사격 조사",
"JKV"	:"호격 조사",
"JKQ"	:"인용격 조사",
"JX"	:"보조사",
"JC"	:"접속 조사",
"EP"	:"선어말 어미",
"EF"	:"종결 어미",
"EC"	:"연결 어미",
"ETN"	:"명사형 전성 어미",
"ETM"	:"관형형 전성 어미",
"XPN"	:"체언 접두사",
"XSN"	:"명사 파생 접미사",
"XSV"	:"동사 파생 접미사",
"XSA"	:"형용사 파생 접미사",
"XSA-R"	:"형용사 파생 접미사(규칙)",
"XSA-I"	:"형용사 파생 접미사(불규칙)",
"XSM"	:"부사 파생 접미사",
"XR"	:"어근",
"SF"	:"종결 부호(. ! ?)",
"SP"	:"구분 부호(, / : ;)",
"SS"	:"""인용 부호 및 괄호(" " ( ) [ ] < > { } ― ‘ ’ “ ” ≪ ≫ 등)""",
"SSO"	:"SS 중 여는 부호",
"SSC"	:"SS 중 닫는 부호",
"SE"	:"줄임표(…)",
"SO"	:"붙임표(- ~)",
"SW"	:"기타 특수 문자",
"SL"	:"알파벳(A-Z a-z)",
"SH"	:"한자",
"SN"	:"숫자(0-9)",
"SB"	:"순서 있는 글머리(가. 나. 1. 2. 가) 나) 등)*",
"UN"	:"분석 불능",
"W_URL"	:"URL 주소",
"W_EMAIL"	:"이메일 주소",
"W_HASHTAG"	:"해시태그(#abcd)",
"W_MENTION"	:"멘션(@abcd)",
"W_SERIAL"	:"일련번호(전화번호, 통장번호, IP주소 등)",
"Z_CODA"	:"덧붙은 받침",
"USER0_4"	:"사용자 정의 태그"}



# 문법 항목 리스트 생성
"""
"/" 로 끊어서 판단.
품사번호 확인 후 형태가 같으면 해당 태그가 사용 됨.

"""
grammatical_items = [
    {
        "번호": 1,
        "형태": '이/가',
        '품사': 'JKS',
        '의미': '어떤 상태를 보이는 대상이나 일정한 상태나 상황을 겪는 경험주 또는 일정한 동작의 주체임을 나타내는 격 조사',
    },
    {
        '번호': 2,
        '형태': '은/는',
        '품사': 'JX',
        '의미': '문장 속에서 어떤 대상이 화제임을 나타내는 보조사',
    },
    {
        '번호': 3,
        '형태': '은/는',
        '품사': 'JX',
        '의미': '어떤 대상이 다른 것과 대조됨을 나타내는 보조사',
    },
    {
        '번호': 4,
        '형태': '은/는',
        '품사': 'JX',
        '의미': '강조의 뜻을 나타내는 보조사',
    },
    {
        '번호': 5,
        '형태': '-은',
        '품사': 'ETM',
        '의미': '앞말이 관형어 구실을 하게 하고 동작이 과거에 이루어졌음을 나타내는 어미',
    },
    {
        '번호': 6,
        '형태': '-는',
        '품사': 'ETM',
        '의미': '앞말이 관형어 구실을 하게 하고 이야기하는 시점에서 볼 때 사건이나 행위가 현재 일어남을 나타내는 어미',
    },
    {
        '번호': 7,
        '형태': '이',
        '품사': 'MM',
        '의미': '말하는 이에게 가까이 있거나 말하는 이가 생각하고 있는 대상을 가리킬 때 쓰는 말',
    },
    {
        '번호': 8,
        '형태': '그',
        '품사': 'MM',
        '의미': '듣는 이에게 가까이 있거나 듣는 이가 생각하고 있는 대상을 가리킬 때 쓰는 말',
    },
    {
        '번호': 9,
        '형태': '저',
        '품사': 'MM',
        '의미': '말하는 이와 듣는 이로부터 멀리 있는 대상을 가리킬 때 쓰는 말',
    },
    {
        '번호': 10,
        '형태': '에',
        '품사': 'JKB',
        '의미': '앞말이 처소의 부사어임을 나타내는 격 조사 또는 진행 방향의 부사어임을 나타내는 격 조사',
    },
    {
        '번호': 11,
        '형태': '에',
        '품사': 'JKB',
        '의미': '앞말이 시간의 부사어임을 나타내는 격 조사',
    },
    {
        '번호': 12,
        '형태': '도',
        '품사': 'JX',
        '의미': '이미 어떤 것이 포함되고 그 위에 더함의 뜻을 나타내는 보조사',
    },
    {
        '번호': 13,
        '형태': '만',
        '품사': 'JX',
        '의미': '다른 것으로부터 제한하여 어느 것을 한정함을 나타내는 보조사',
    },
    {
        '번호': 14,
        '형태': '다',
        '품사': 'EF',
        '의미': '해라할 자리에 쓰여, 어떤 사건이나 사실, 상태를 서술하는 뜻을 나타내는 종결 어미',
    },
    {
        '번호': 15,
        '형태': '다',
        '품사': 'MAG',
        '의미': '남거나 빠진 것이 없이 모두',
    },
    {
        '번호': 16,
        '형태': '보고',
        '품사': 'JKB',
        '의미': '어떤 행동이 미치는 대상임을 나타내는 격 조사',
    },
    {
        '번호': 17,
        '형태': '-(으)시-/으/시/셧/세/세요/으시',
        '품사': 'EP',
        '의미': '높임의 뜻을 더하는 선어말 어미',
    },
    {
        '번호': 18,
        '형태': '-아/어/여/아',
        '품사': 'EF',
        '의미': '해할 자리에 쓰여, 어떤 사실을 서술하거나 물음ㆍ명령ㆍ청유를 나타내는 종결 어미',
    },
    {
        '번호': 19,
        '형태': '(으)니까/으니/으니까/니까',
        '품사': 'EC',
        '의미': '앞말이 뒷말의 원인이나 근거, 전제 따위가 됨을 나타내는 연결 어미',
    },
    {
        '번호': 20,
        '형태': '들',
        '품사': 'XSN',
        '의미': '복수의 뜻을 더하는 접미사',
    },
    {
        '번호': 21,
        '형태': '(이)나/이나/나',
        '품사': 'JX',
        '의미': '둘 이상의 사물을 같은 자격으로 이어 주는 접속 조사',
    },
    {
        '번호': 22,
        '형태': '(이)나/이나/나',
        '품사': 'JX',
        '의미': '수량이 크거나 많음, 혹은 정도가 높음을 강조하는 보조사',
    },
    {
        '번호': 23,
        '형태': '(이)나/나/이나',
        '품사': 'JX',
        '의미': '마음에 차지 않는 선택, 또는 최소한 허용되어야 할 선택이라는 뜻을 나타내는 보조사',
    },
    {
        '번호': 24,
        '형태': '밖에',
        '품사': 'JX',
        '의미': '그것 말고는, 그것 이외에는의 뜻을 나타내는 보조사',
    },
    {
        '번호': 25,
        '형태': '안',
        '품사': 'MAG',
        '의미': '부정이나 반대의 뜻을 나타내는 말',
    },
    {
        '번호': 26,
        '형태': '안',
        '품사': 'NNG',
        '의미': '어떤 물체나 공간의 둘러싸인 가에서 가운데로 향한 쪽. 또는 그런 곳이나 부분',
    },
    {
        '번호': 27,
        '형태': '-게',
        '품사': 'EC',
        '의미': '앞의 내용이 뒤에서 가리키는 사태의 목적이나 결과, 방식, 정도 따위가 됨을 나타내는 연결 어미',
    },
    {
        '번호': 99,
        '형태': '에',
        '품사': 'JKB',
        '의미': '앞의 체언을 부사어가 되게 하는 조사',
    },
]

# 로직컬 적으로 판단을 위해
 
# 시간 명사
time_nouns = {
"시간", "때", "시각", "분", "초", "어제", "오늘", "내일", "모레", "그제", "글피", "지난주", "이번주", "다음주",
"월요일", "화요일", "수요일", "목요일", "금요일", "토요일", "일요일", "주말", "평일", "아침", "점심", "저녁", "밤", "새벽", "낮",
"방금", "그때", "이때", "저때", "지금", "그동안", "이후", "이전", "지난달", "이번달", "다음달", "작년", "올해", "내년",
"봄", "여름", "가을", "겨울", "어린 시절", "청소년기", "성인기", "노년기", "초", "중", "말", "첫날", "마지막 날",
"전날", "이튿날", "사흘", "나흘", "닷새", "엿새", "일주일", "보름", "한 달", "두 달", "세 달", "석 달", "반년", "일년", "두 해", "세 해",
"연초", "연말", "중순", "상반기", "하반기", "사계절", "1분기", "2분기", "3분기", "4분기", "10년", "세기", "밀레니엄", "대공황",
"유년기", "사춘기", "청년기", "중년기", "노년기", "이전", "다음", "이후", "현재", "과거", "미래", "선사시대", "고대", "중세", "근세", "현대",
"새벽 1시", "새벽 2시", "오전 3시", "오전 4시", "오후 5시", "오후 6시", "밤 7시", "밤 8시", "자정", "정오", 
"초반", "중반", "후반", "초기", "중기", "말기", "하루", "주중", "주간", "연중", "평생", "영원", "순간", "찰나", "잠시", "얼마간", 
"하루하루", "해마다", "계속", "영겁", "어느 날", "어느 순간", "동이 틀 무렵", "해질 녘", "밤중", "한낮", "정오", "이른 아침", "늦은 저녁",
"초저녁", "심야", "한밤중", "말년", "초창기", "말미", "막바지", "오랜만", "잠깐", "순식간", "한순간", "짧은 시간", "긴 시간",
"첫 번째", "두 번째", "세 번째", "네 번째", "다섯 번째", "처음", "마지막", "종반", "결말", "피날레",
"20세기", "21세기", "100년", "200년", "천년", "십년", "반세기", "한 세대", "두 세대", "역사 속에서", "오래전", 
"아득한 옛날","시간", "때", "시각", "분", "초", "어제", "오늘", "내일", "모레", "지난주", "이번주", "다음주",
"월요일", "화요일", "수요일", "목요일", "금요일", "토요일", "일요일", "주말", "아침", "점심", "저녁", "밤", "새벽", "방금", "그때", "지난달",
"이번달", "다음달", "작년", "올해", "내년", "저번" , "이번", "옛날", "새벽","아침","오전","자정","주","오전","백년","십년","일년",
"오후","오후 1시","오후 2시","오후 3시","오후 4시","오후 5시","오후 6시","오후 7시","오후 8시","오후 9시","오후 10시","오후 11시","오후 12시",
"오전","오전 1시","오전 2시","오전 3시","오전 4시","오전 5시","오전 6시","오전 7시","오전 8시","오전 9시","오전 10시","오전 11시","오전 12시",
"밤","밤 1시","밤 2시","밤 3시","밤 4시","밤 5시","밤 6시","밤 7시","밤 8시","밤 9시","밤 10시","밤 11시","밤 12시",
"낮","낮 1시","낮 2시","낮 3시","낮 4시","낮 5시","낮 6시","낮 7시","낮 8시","낮 9시","낮 10시","낮 11시","낮 12시",
"봄에","여름에","가을에","겨울에","초봄에","늦가을에","초여름에","한여름에","연말에","연초에","연중에","계절에","새벽에","아침에","점심에","저녁에","밤에","자정에",
"정오에","해질녘에","동틀 무렵에","오후에","이른 아침에","한낮에","늦은 밤에","내일","모레","다음 주에","다음 달에","내년에","미래에","앞으로","곧","장차","머지않아",
"훗날에","언젠가","장래에","이후에","곧바로","차후에","나중에","몇 년 후에","먼 훗날에","지금","현재","이 시간에","오늘","요즘에","요새","금방","방금","당장","오늘 아침에",
"이번 주에","올해","이번 달에","이번 학기에","이번에","이번 시기에","최근에","옛날","옛적에","예전에","과거에","지난달에","작년에","어제","그제","지난주에","한참 전에","몇 년 전에",
"어릴 때","유년 시절에","어린 시절에","사춘기 때","중학교 때","고등학교 때","대학교 때","어렸을 때","이전에","과거 어느 날","한때","초창기에","초기에","아득한 옛날에"
}

# 수량 명사
quantity_nouns = {
    '한', '두', '세', '네', '다섯', '여섯', '일곱', '여덟', '아홉', '열', '백', '천', '만',
    '개', '명', '번', '시간', '일', '주', '달', '년', '원', '킬로그램', '그램', '미터', '센티미터', '리터', '도'
}

place_nouns = {
    # 장소 명사
    "집", "학교", "회사", "공원", "도서관", "식당", "병원", "은행", "시장", "백화점",
    "마트", "극장", "역", "공항", "호텔", "카페", "레스토랑", "교실", "방", "거리",
    "도로", "바다", "산", "섬", "도시", "나라", "지하철", "버스정류장", "주차장",
    "화장실", "사무실", "교회", "성당", "절", "박물관", "미술관", "경찰서", "소방서",
    "체육관", "운동장", "수영장", "고속도로", "터미널", "아파트", "빌딩", "건물",
    "영화관", "놀이공원", "공연장", "전시장", "서점", "약국", "미용실", "음식점",
    "카센터", "주유소", "헬스장", "노래방", "카페테리아", "식품점", "편의점", "슈퍼마켓",
    "학교", "회사", "집", "교실", "강의실", "연구실", "실험실", "운전면허시험장",
    "운전학원", "운동장", "놀이방", "어린이집", "유치원", "초등학교", "중학교", "고등학교", "대학교","시골"
    "집","학교","회사","공원","병원","도서관","식당","극장","서점","카페","시장","백화점","마트","공장","경찰서","소방서","은행","우체국","박물관","미술관","체육관","공항","호텔","산","바다","강","호수","숲","해변","사막","동굴","계곡","들판","섬","폭포","초원","역","버스 정류장","지하철역","터미널","항구","주차장","정류장","고속도로","톨게이트",
    "아파트","빌딩","교회","성당","사찰","궁전","성","운동장","야구장","축구장","수영장","놀이공원","경기장",
    "한국","미국","일본","중국","영국","서울","부산","뉴욕","런던","파리","베이징","도쿄","베를린","로마","시드니",
    "화장실","교실","강의실","운전면허시험장","회의실","사무실","운전석","부엌","거실","방",

    # 진행 방향을 나타내는 명사
    "앞", "뒤", "옆", "위", "아래", "안", "밖", "사이", "중간", "주변", "근처",
    "오른쪽", "왼쪽", "동쪽", "서쪽", "남쪽", "북쪽", "내부", "외부", "방향", "쪽",
    "방면", "코너", "목적지", "출발지", "도착지", "지점", "위치", "장소", "경로",
    "길", "골목", "입구", "출구", "교차로", "사거리", "삼거리", "기차역", "버스터미널",
    "항구", "맞은편", "정면", "후면", "상단", "하단", "중앙", "가운데", "주위", "둘레",
    "바깥", "안쪽", "외곽", "부근", "접근", "도로", "차선", "차로", "이정표", "교차점",
    "코앞", "뒤편", "옆쪽", "바로", "곧장", "직진", "우회전", "좌회전", "유턴", "골목길",
    "샛길", "지름길", "고가도로", "지하도", "육교", "다리", "터널", "고개", "언덕","쪽"
}

In [2]:
kiwi.analyze("테스트 텍스트 입니다.")

[([Token(form='테스트', tag='NNG', start=0, len=3),
   Token(form='텍스트', tag='NNG', start=4, len=3),
   Token(form='이', tag='VCP', start=8, len=1),
   Token(form='ᆸ니다', tag='EF', start=8, len=3),
   Token(form='.', tag='SF', start=11, len=1)],
  -33.78171157836914)]

In [3]:
# 함수 내에서 동기적으로 호출
def chatgpt(text, matched_items, pna_token, now_token):
    response = openai.ChatCompletion.create(
        model="gpt-4", #gpt-4 ,gpt-4o-mini , gpt-4o, gpt-3.5-turbo
        messages=[{
            "role": "user",
            "content": f"""
            sentence: {text}
            pna_token : {pna_token}
            now_token : {now_token}
            matched_items: {matched_items}

            현재 나는 한국어 문장 태깅에서 판단하기 어려운 부분을 너에게 맡기려고 해.
            sentence는 전체 문장이고, pna_token의 가운데인 now_token의 태깅을 판단해야해.
            matched_items는 현재 태깅의 어려운 부분이야. 이 부분이 2가지 이상의 태깅이 되어있어.
            matched_items의 번호가 몇번이 정답일지 판단해서 숫자로 정답만 알려줘
            matched_items의 번호가 0번인 경우 nlp모델이 분류한 결과야 이게 맞을 수 있는데 나머지 번호가 틀렸다고 판단될 경우 0번으로 알려줘
            그렇지 않은 경우 반드시 matched_items의 번호 중에서의 숫자만 알려줘.
            반드시 설명하지말고 숫자만 알려줘.
            """
        }],
        stream=False,  # 스트리밍 비활성화
    )

    # 응답 처리
    full_text = str(response['choices'][0]['message']['content'])
    print("====GPT answer====", full_text)

    return full_text

In [4]:
grammatical_items_dict = {item['번호']: item for item in grammatical_items} # 딕셔너리로 빨리 찾기
# token_1 = grammatical_items_dict[n] # n번 태깅 찾기

In [5]:
change_pos_descriptions = [grammatical_items[k] for k in range(len(grammatical_items)) if (grammatical_items[k]['번호']==16)][0] # 16번 태깅 찾기
grammatical_items_dict[16] # n번 태깅 찾기


{'번호': 16, '형태': '보고', '품사': 'JKB', '의미': '어떤 행동이 미치는 대상임을 나타내는 격 조사'}

In [6]:
import tiktoken

# 사용하는 모델에 맞는 인코더 선택 (예: gpt-3.5-turbo)
encoding = tiktoken.get_encoding("cl100k_base")

# 텍스트를 토큰으로 변환
text = "여기에 계산하고 싶은 텍스트를 입력하세요."
tokens = encoding.encode(text)

# 토큰의 개수 출력
print(len(tokens))

17


In [7]:
# #############
# # 
# def pos_tag_print(results):
#     output = []
#     # 형태소 분석 결과와 문법 항목 매핑
#     for tokens, score in results:
#         for token in tokens:
#             morpheme = token.form  # 형태소 (form)
#             pos = token.tag        # 품사 태그 (tag)
#             pos_desc = pos_descriptions.get(pos, '알 수 없음')
#             matched_items = []
#             for item in grammatical_items:
#                 # 형태에서 '-'를 제거하고 '/'로 분리하여 각 형태소를 비교
#                 item_forms = item['형태'].replace('-', '').split('/')
#                 if morpheme in item_forms and pos == item['품사']:
#                     matched_items.append(item)
#             output.append({
#                 'morpheme': morpheme,
#                 'pos': pos,
#                 'pos_desc': pos_desc,
#                 'matched_items': matched_items
#             })
#     return output

def pos_tag_print(results):
    output = []
    # 형태소 분석 결과와 문법 항목 매핑
    for tokens, score in results:
        for token in tokens:
            morpheme = token.form  # 형태소 (form)
            pos = token.tag        # 품사 태그 (tag)
            start = token.start    # 형태소의 시작 인덱스
            length = token.len     # 형태소의 길이
            end = start + length   # 형태소의 끝 인덱스 계산
            pos_desc = pos_descriptions.get(pos, '알 수 없음')
            matched_items = []
            for item in grammatical_items:
                # 형태에서 '-'를 제거하고 '/'로 분리하여 각 형태소를 비교
                item_forms = item['형태'].replace('-', '').split('/')
                if morpheme in item_forms and pos == item['품사']:
                    matched_items.append(item)
            output.append({
                'morpheme': morpheme,
                'pos': pos,
                'pos_desc': pos_desc,
                'matched_items': matched_items,
                'start': start,
                'len': length,
                'end': end
            })
    return output

In [17]:


#문장을 끊어서 판단
def split_sentences(output): 
    sentences = []
    sentence = []
    for token in output: 
        sentence.append(token)
        if token['pos'] in ['SF']: # 문장 구분: 종결 어미(EF)와 종결 부호(SF)를 기준으로 문장을 분리 # @@ 바꾸어야 할 지도.
            sentences.append(sentence)
            sentence = []
    # 마지막 문장이 EF나 SF로 끝나지 않는 경우 처리
    if sentence:
        sentences.append(sentence)
    return sentences


def check_logic(sentence_tokens):
    
    output = all_text = sentence_tokens # GPT에 쓰임
    assigned_indices = set()
    
    ### 은/는 JX 처리
    # 문장 내에서 '은/는'이고 품사가 'JX'인 형태소의 인덱스 목록 추출
    eunnun_indices = [i for i, token in enumerate(output) if token['morpheme'] in ['은', '는'] and token['pos'] == 'JX']

    # 3번 항목 처리: '은/는'이 두 번 이상 등장하고, 그 사이에 EC가 있으며,
    # 두 '은/는'이 같은 글자이고, 그 앞의 명사가 같은 태깅인 경우
    # between_tokens에 명사가 있더라도, 그 명사가 EC 뒤에 오고 SP 앞에 오는 경우에는 넘어가지 않음
    
    if len(eunnun_indices) >= 2:
        for idx1 in eunnun_indices:
            for idx2 in eunnun_indices:
                if idx2 <= idx1:
                    continue
                token1 = output[idx1]
                token2 = output[idx2]
                
                # '은'과 '는'이 같은 글자인 경우만 처리
                # if token1['morpheme'] != token2['morpheme']:
                #     continue  # '은'과 '는'이 다르면 넘어감
                
                # '은'이나 '는' 앞의 명사를 확인
                if idx1 > 0 and idx2 > 0:
                    prev_token1 = output[idx1 - 1]
                    prev_token2 = output[idx2 - 1]
                    # 앞의 토큰이 명사인지 확인
                    if not prev_token1['pos'].startswith('N') or not prev_token2['pos'].startswith('N'):
                        continue  # 둘 중 하나라도 명사가 아니면 넘어감
                    # 명사의 태깅이 같은지 확인
                    # if prev_token1['pos'] != prev_token2['pos']:
                    #     continue  # 명사의 품사 태그가 다르면 넘어감
                else:
                    continue  # 앞의 토큰이 없으면 넘어감
                
                # 두 '은/는' 사이의 토큰들 추출
                between_tokens = output[idx1+1:idx2]
                
                # between_tokens에 EC와 SP가 있는지 확인
                has_EC = any(tok['pos'] == 'EC' for tok in between_tokens)
                has_SP = any(tok['pos'] == 'SP' for tok in between_tokens)
                if not (has_EC and has_SP):
                    continue  # EC와 SP가 모두 없으면 넘어감
                
                # 조건을 모두 만족하면 해당 인덱스에 번호 3번 항목 할당
                for idx in [idx1, idx2]:
                    print("모든 조건 만족")
                    token = output[idx]
                    item3 = next((item for item in token['matched_items'] if item['번호'] == 3), None)
                    if item3:
                        token['matched_items'] = [item3]
                        assigned_indices.add(idx)  # 태깅된 인덱스 저장
                        
    # 4번 항목 처리: '은/는' 뒤에 동사(VV)가 오는 경우
    for idx in eunnun_indices:
        if idx in assigned_indices:
            continue  # 이미 태깅된 경우 건너뜀
        # 강조가 아니라고 판단 될 경우
        token = output[idx]
        if idx + 1 < len(output):
            next_token = output[idx + 1]
            if next_token['pos'] in ['VV', 'VA']:
                item4 = next((item for item in token['matched_items'] if item['번호'] == 4), None)
                
                if item4:
                    token['matched_items'] = [item4]
                    assigned_indices.add(idx)
                continue
            
                            
    # 2번 항목 처리: 앞에 명사(N*)가 오는 경우
    for idx in eunnun_indices:
        if idx in assigned_indices:
            continue  # 이미 태깅된 경우 건너뜀
        token = output[idx]
        assigned = False
        if idx > 0:
            prev_token = output[idx - 1]
            if prev_token['pos'].startswith('N'):
                item2 = next((item for item in token['matched_items'] if item['번호'] == 2), None)
                
                if item2:
                    token['matched_items'] = [item2]
                    assigned_indices.add(idx)
                    assigned = True
                    
        if not assigned and idx >= 2:
            prev_prev_token = output[idx - 2]
            prev_token = output[idx - 1]
            if prev_prev_token['pos'].startswith('N') and prev_token['pos'] in ['JKB','JKS' ,'XSN']:
                item2 = next((item for item in token['matched_items'] if item['번호'] == 2), None)
                if item2:
                    token['matched_items'] = [item2]
                    assigned_indices.add(idx)
                        
        # 새로운 로직 추가: 이전 이전 토큰이 명사(N*), 이전 토큰이 XSN 또는 JKB인 경우 , 이전 토큰이 명사 인 경우
        if not assigned and idx >= 2:
            prev_prev_token = output[idx - 2]
            prev_token = output[idx - 1]
            
            if prev_prev_token['pos'].startswith('N') and prev_token['pos'] in ['XSN', 'JKB']:
                # 번호 2로 설정
                # print("이전 이전 토큰이 명사(N*), 이전 토큰이 XSN 또는 JKB인 경우=========")
                item2 = next((item for item in token['matched_items'] if item['번호'] == 2), None)
                if item2:
                    token['matched_items'] = [item2]
                    assigned = True
                    
    # # 4번 항목 처리: '은/는' 뒤에 동사(VV)가 오는 경우
    for idx in eunnun_indices:
        if idx in assigned_indices:
            continue  # 이미 태깅된 경우 건너뜀
        ## @@ 나머지 전부 강조??
        token = output[idx]
        item4 = next((item for item in token['matched_items'] if item['번호'] == 4), None)
        if item4:
            token['matched_items'] = [item4]
            assigned_indices.add(idx)
        continue

        ##
        
        # # 강조가 아니라고 판단 될 경우
        # token = output[idx]
        # if idx + 1 < len(output):
        #     next_token = output[idx + 1]
        #     if next_token['pos'] == 'VV':
        #         item4 = next((item for item in token['matched_items'] if item['번호'] == 4), None)
                
        #         if item4:
        #             token['matched_items'] = [item4]
        #             assigned_indices.add(idx)
        #         continue
                    
    ### JKB 처리 (10번 11번)
    for idx, token in enumerate(output):
        if token['morpheme'] == '에' and token['pos'] == 'JKB':
            assigned = False  # 번호가 할당되었는지 여부를 추적
            # 앞의 토큰을 가져옵니다.
            if idx > 0:
                # '에' 앞의 가장 가까운 명사(N)를 찾아 시간 표현인지 확인
                for back_idx in range(idx - 1, -1, -1):
                    back_token = output[back_idx]
                    # print(back_token)  # 디버깅용 출력
                    if back_token['pos'].startswith('N'):
                        print(back_token['morpheme'])  # 디버깅용 출력
                        
                        
                        if back_token['morpheme'] in time_nouns:
                            print("11번", back_token['morpheme'])  # 디버깅용 출력
                            # 시간 표현 단어를 찾았으므로 번호 11로 설정
                            item11 = next((item for item in token['matched_items'] if item['번호'] == 11), None)
                            if item11:
                                token['matched_items'] = [item11]
                                break  # 가장 가까운 N*를 찾았으므로 루프 종료

                        if back_token['morpheme'] in place_nouns:
                            print("10번", back_token['morpheme'])  # 디버깅용 출력
                            # 처소 및 진행 방향의 부사어 인지 판단하는 로직 추가
                            item10 = next((item for item in token['matched_items'] if item['번호'] == 10), None)
                            if item10:
                                token['matched_items'] = [item10]
                                break  # 가장 가까운 N*를 찾았으므로 루프 종료
                        else:
                            # 가장 가까운 명사가 시간이나 장소가 아니니 태깅 x
                            break
            else:
                # 앞의 토큰이 없는 경우 번호를 할당하지 않음
                pass
            
                    
    ### '(이)나' (JX) 처리 22,23,21도
    # 문장 내에서 '(이)나' (JX)를 처리
    for idx, token in enumerate(output):
        if token['morpheme'] in ['이나', '나'] and token['pos'] == 'JX':
            assign_number = None  # 22 또는 23으로 설정할 변수

            # 조건 1: 수사(NR) 또는 숫자(SN) + 명사(N*) + JX('이나'/'나') (번호 22번)
            if idx >= 2:
                prev_token = output[idx - 1]
                prev_prev_token = output[idx - 2]
                if (prev_prev_token['pos'] in ['NR', 'SN']) and prev_token['pos'].startswith('N'):
                    assign_number = 22

            # 조건 2: MM(수량 관형사) + 명사(N*) + JX('이나'/'나') (번호 22번)
            if assign_number is None and idx >= 2:
                prev_token = output[idx - 1]
                prev_prev_token = output[idx - 2]
                if (prev_prev_token['pos'] == 'MM' and prev_prev_token['morpheme'] in quantity_nouns and prev_token['pos'].startswith('N')):
                    assign_number = 22

            # 조건 3: EC + NNG + JX('이나'/'나') (번호 23번)
            if assign_number is None and idx >= 2:
                prev_token = output[idx - 1]
                prev_prev_token = output[idx - 2]
                if prev_prev_token['pos'] == 'EC' and prev_token['pos'] == 'NNG':
                    assign_number = 23

            # 조건 4: JX('이나'/'나') 뒤에 VV (번호 23번)
            if assign_number is None and idx + 1 < len(output):
                next_token = output[idx + 1]
                # if next_token['pos'] == 'VV':
                if next_token['pos'].startswith('VV'):
                    assign_number = 23
                    
            # 조건 5: JX('이나'/'나') 사이에 N*  (번호 21번)
            if assign_number is None:
                # 앞뒤 토큰을 확인합니다.
                prev_token = output[idx - 1] if idx > 0 else None
                next_token = output[idx + 1] if idx + 1 < len(output) else None
                nnext_token = output[idx + 2] if idx + 2 < len(output) else None
                
                if prev_token and next_token:
                    # 앞뒤 토큰의 품사가 명사(N*)인지 확인
                    if prev_token['pos'].startswith('N') and next_token['pos'].startswith('N') and (nnext_token['pos'].endswith('V') == False ):
                        assign_number = 21

            # 조건 6: JX('이나'/'나') 사이에 N*  (번호 21번) => 현재 이렇게 해도 되나 .. 싶음
            if assign_number is None:
                prev_token = output[idx - 1] if idx > 0 else None
                next_token = output[idx + 1] if idx + 1 < len(output) else None

                if prev_token and next_token:
                    # 앞뒤 토큰의 품사가 명사(N*)인지 확인
                    if prev_token['pos'].startswith('N') and next_token['pos'].startswith('N'):
                        assign_number = 21
                        
            # 번호 할당
            if assign_number == 22:
                item22 = next((item for item in token['matched_items'] if item['번호'] == 22), None)
                if item22:
                    token['matched_items'] = [item22]
                    
            elif assign_number == 23:
                item23 = next((item for item in token['matched_items'] if item['번호'] == 23), None)
                if item23:
                    token['matched_items'] = [item23]
                    
            elif assign_number == 21:
                item21 = next((item for item in token['matched_items'] if item['번호'] == 21), None)
                if item21:
                    token['matched_items'] = [item21]    


            else:
                # 조건에 해당하지 않는 경우 matched_items를 그대로 유지
                pass

    ### '나/이나' (JC) 처리: 아이템 21번 태깅
    for idx, token in enumerate(output):
        if token['morpheme'] in ['나', '이나'] and token['pos'] == 'JC':
            # 앞뒤 토큰을 확인합니다.
            prev_token = output[idx - 1] if idx > 0 else None
            next_token = output[idx + 1] if idx + 1 < len(output) else None
            nnext_token = output[idx + 2] if idx + 2 < len(output) else None

            if prev_token and next_token:
                # 앞뒤 토큰의 품사가 명사(N*)인지 확인
                if prev_token['pos'].startswith('N') and next_token['pos'].startswith('N') and (nnext_token['pos'].endswith('V') == False):
                    # 번호 21번 항목을 가져옵니다.
                    token_1 = grammatical_items_dict[21] # n번 태깅 찾기
                    token['pos']= token_1["품사"]
                    token['pos_desc']= token_1["의미"]
                    token['matched_items'].append(token_1)
        
    ### '세요' (EF) 처리  17번 (강제로 찾게 만듦.)
    for token in output:
        if token['pos'] == 'EF' and token['morpheme'].endswith('세요'):

            token_1 = token_1 = grammatical_items_dict[17] # n번 태깅 찾기
            token['pos']= token_1["품사"]
            token['pos_desc']= token_1["의미"]
            token['matched_items'].append(token_1)
    
    # '수' + '밖' + '에' 패턴 병합 로직 추가
    idx = 0
    while idx < len(output) - 2:
        prev_token = output[idx]
        token = output[idx + 1]
        next_token = output[idx + 2]
        # '수'(NNB) + '밖'(NNG) + '에'(JKB) 패턴 확인
        if (prev_token['morpheme'] == '수' and prev_token['pos'] == 'NNB' and
            token['morpheme'] == '밖' and token['pos'] == 'NNG' and
            next_token['morpheme'] == '에' and next_token['pos'] == 'JKB'):
            # '밖' + '에'를 병합하여 '밖에'로 만듦
            merged_token = {
                'morpheme': token['morpheme'] + next_token['morpheme'],
                'pos': token_1["품사"],
                'pos_desc': token_1["의미"],
                'matched_items': [token_1],
                'start': token['start'],
                'len': token['len'] + next_token['len'],
                'end': token['start'] + token['len'] + next_token['len']
            }
            # 문법 항목 번호 24번 적용
            item24 = grammatical_items_dict[24]  # 번호 24번 항목
            if item24:
                merged_token['matched_items'].append(item24)
            # '수'는 그대로 두고 '밖'과 '에'를 병합된 토큰으로 대체
            output = output[:idx + 1] + [merged_token] + output[idx + 3:]
            # 인덱스 조정
            idx += 1
        else:
            idx += 1

    # # '밖' + '에' 패턴 처리: 앞에 '수'가 없는 경우
    # idx = 0
    # while idx < len(output) - 1:
    #     token = output[idx]
    #     next_token = output[idx + 1]
    #     # '밖'(NNG) + '에'(JKB) 패턴 확인
    #     if (token['morpheme'] == '밖' and token['pos'] == 'NNG' and
    #         next_token['morpheme'] == '에' and next_token['pos'] == 'JKB'):
    #         # '에'에 문법 항목 번호 10번 적용
    #         item10 = grammatical_items_dict[10]  # 번호 10번 항목
    #         if item10:
    #             next_token['matched_items'] = [item10]
    #         idx += 1
    #     else:
    #         idx += 1
            
    ### 16번 태깅 '보' + '고' 패턴 병합 로직 추가
    idx = 0
    while idx < len(output) - 1:
        token = output[idx]
        next_token = output[idx + 1]
        if token['morpheme'] == '보' and token['pos'] == 'VV' and next_token['morpheme'] == '고' and next_token['pos'] == 'EC':
            token_1 = grammatical_items_dict[16] # n번 태깅 찾기
            
            print(token_1)
            # '보' + '고'를 병합하여 '보고'로 만듦
            merged_token = {
                'morpheme': token['morpheme'] + next_token['morpheme'],
                'pos': token_1["품사"],
                'pos_desc': token_1["의미"],
                'matched_items': [token_1],
                'start': token['start'],
                'len': token['len'] + next_token['len'],
                'end': token['start'] + token['len'] + next_token['len']
            }
            # 기존 토큰을 삭제하고 병합된 토큰을 삽입
            output = output[:idx] + [merged_token] + output[idx + 2:]
            # 인덱스 조정
            if idx > 0:
                idx -= 1  # 병합 후 이전 위치로 이동하여 앞의 토큰과의 관계도 검사
            else:
                idx = 0
        else:
            idx += 1
            
    # N* N* N* 패턴에서 가운데 N이 '보고'인 경우 번호 16번 태깅 (강제로 찾게 만듦.)
    for idx in range(1, len(output) - 1):
        token = output[idx]
        if token['pos'] == 'NNG' and token['morpheme'] == '보고':
            prev_token = output[idx - 1]
            next_token = output[idx + 1]
            if prev_token['pos'].startswith('N') and next_token['pos'].startswith('N'):
                # 번호 16으로 설정
                token_1 = grammatical_items_dict[16] # n번 태깅 찾기
                token['pos']= token_1["품사"]
                token['pos_desc']= token_1["의미"]
                token['matched_items'].append(token_1)


    # gpt 이용 부문.
    # @@ 여기서 gpt가 답을 고르지 못한 경우 None 태그를 만들어서 이 경우엔 기본 태그가 나오게 하자.
    # 최종, 태그가 2개 이상 있는 경우 해당 태그를 llm모델에 맡겨서 뭐가 맞는지 판단하게 함.
    for idx, token in enumerate(output):
        if len(token["matched_items"]) >= 2:
            # 이전 토큰과 다음 토큰을 가져옵니다.
            pre_token = output[idx - 1]["morpheme"] if idx > 0 else ''
            now_token = token["morpheme"]
            after_token = output[idx + 1]["morpheme"] if idx + 1 < len(output) else ''

            # 문장 전체 텍스트를 재구성합니다.
            sentence_text = ''.join(tok['morpheme'] for tok in output)

            try:
                # GPT 함수 호출
                token['matched_items'].insert(0,{'번호': 0,'형태': token['morpheme'],'품사': token['pos'],'의미': token['pos_desc']}) # 기존 형태의 품사(nlp가 분류한 더 큰 집합[분류]의 품사)를 넣고 비교
                print(token['matched_items'])
                gpt_answer = chatgpt(sentence_text, token["matched_items"], pre_token + now_token + after_token, now_token)
                
                correct_tag = int(re.findall(r'\d+', gpt_answer)[0]) # GPT 응답에서 숫자 추출
                print(correct_tag,"correct_tag")

                item = next((item for item in token['matched_items'] if item['번호'] == correct_tag), None)
                if item:
                    token['matched_items'] = [item]

                print("llm사용됨", sentence_text)
                print("정답으로 만든 태그", correct_tag, token["matched_items"])
                
            except Exception as e:
                correct_tag = 0
                print("GPT오류", e)
                pass    
            
            if correct_tag == 0: # 0번의 경우 그대로 태깅
                token['matched_items'] = []
            else:
                item = next((item for item in token['matched_items'] if item['번호'] == correct_tag), None)
                if item:
                    token['matched_items'] = [item]
            
        else:
            # 태그가 1개인 경우 패스 
            pass
        
    # 최종 결과 출력
    for token in output:
        morpheme = token['morpheme']
        pos = token['pos']
        pos_desc = token['pos_desc']
        start = token['start']
        length = token['len']
        matched_items = token['matched_items']

        if matched_items and (matched_items[0]['번호'] != 0):
            for item in matched_items:
                print(f"형태소 '{morpheme}' (위치: {start}, 길이: {length}, 품사: {pos} - {pos_desc})는 문법 항목 번호 {item['번호']}에 해당합니다: {item['의미']}")
        else:
            print(f"형태소 '{morpheme}' (위치: {start}, 길이: {length}, 품사: {pos} - {pos_desc})")
            
    return output  # 수정된 output 리스트 반환"""

def main(text):
    # pos_tag_print 함수 호출 # 형태소 분석 수행
    output = pos_tag_print(results = kiwi.analyze(text))
    # 문장 단위로 분리 @@ => 이붑분 kiwi 분석기가 있다했음
    
    sentences = split_sentences(output)
    # 각 문장별로 처리하고 결과를 합침

    final_output = []
    for sentence_tokens in sentences:
        processed_sentence = check_logic(sentence_tokens)
        final_output.extend(processed_sentence)
        
    return final_output


In [18]:
# 잘 틀리는 것 예시 잘 못하는

text = """
밖에 덥으니 집에서나 쉬자.
인터넷이 안 되니 책이나 볼 수밖에 없네.
"""

final_output = main(text)
final_output

밖
10번 밖
형태소 '밖' (위치: 1, 길이: 1, 품사: NNG - 일반 명사)
형태소 '에' (위치: 2, 길이: 1, 품사: JKB - 부사격 조사)는 문법 항목 번호 10에 해당합니다: 앞말이 처소의 부사어임을 나타내는 격 조사 또는 진행 방향의 부사어임을 나타내는 격 조사
형태소 '덥' (위치: 4, 길이: 1, 품사: VA-I - 형용사(불규칙))
형태소 '으니' (위치: 5, 길이: 2, 품사: EC - 연결 어미)는 문법 항목 번호 19에 해당합니다: 앞말이 뒷말의 원인이나 근거, 전제 따위가 됨을 나타내는 연결 어미
형태소 '집' (위치: 8, 길이: 1, 품사: NNG - 일반 명사)
형태소 '에서' (위치: 9, 길이: 2, 품사: JKB - 부사격 조사)
형태소 '나' (위치: 11, 길이: 1, 품사: JX - 보조사)는 문법 항목 번호 23에 해당합니다: 마음에 차지 않는 선택, 또는 최소한 허용되어야 할 선택이라는 뜻을 나타내는 보조사
형태소 '쉬' (위치: 13, 길이: 1, 품사: VV - 동사)
형태소 '자' (위치: 14, 길이: 1, 품사: EF - 종결 어미)
형태소 '.' (위치: 15, 길이: 1, 품사: SF - 종결 부호(. ! ?))
형태소 '인터넷' (위치: 17, 길이: 3, 품사: NNG - 일반 명사)
형태소 '이' (위치: 20, 길이: 1, 품사: JKS - 주격 조사)는 문법 항목 번호 1에 해당합니다: 어떤 상태를 보이는 대상이나 일정한 상태나 상황을 겪는 경험주 또는 일정한 동작의 주체임을 나타내는 격 조사
형태소 '안' (위치: 22, 길이: 1, 품사: MAG - 일반 부사)는 문법 항목 번호 25에 해당합니다: 부정이나 반대의 뜻을 나타내는 말
형태소 '되' (위치: 24, 길이: 1, 품사: VV - 동사)
형태소 '니' (위치: 25, 길이: 1, 품사: EC - 연결 어미)
형태소 '책' (위치: 27, 길이: 1, 품사: NNG - 일반 명사)
형태소 '이나' (위치: 28, 길이:

[{'morpheme': '밖',
  'pos': 'NNG',
  'pos_desc': '일반 명사',
  'matched_items': [],
  'start': 1,
  'len': 1,
  'end': 2},
 {'morpheme': '에',
  'pos': 'JKB',
  'pos_desc': '부사격 조사',
  'matched_items': [{'번호': 10,
    '형태': '에',
    '품사': 'JKB',
    '의미': '앞말이 처소의 부사어임을 나타내는 격 조사 또는 진행 방향의 부사어임을 나타내는 격 조사'}],
  'start': 2,
  'len': 1,
  'end': 3},
 {'morpheme': '덥',
  'pos': 'VA-I',
  'pos_desc': '형용사(불규칙)',
  'matched_items': [],
  'start': 4,
  'len': 1,
  'end': 5},
 {'morpheme': '으니',
  'pos': 'EC',
  'pos_desc': '연결 어미',
  'matched_items': [{'번호': 19,
    '형태': '(으)니까/으니/으니까/니까',
    '품사': 'EC',
    '의미': '앞말이 뒷말의 원인이나 근거, 전제 따위가 됨을 나타내는 연결 어미'}],
  'start': 5,
  'len': 2,
  'end': 7},
 {'morpheme': '집',
  'pos': 'NNG',
  'pos_desc': '일반 명사',
  'matched_items': [],
  'start': 8,
  'len': 1,
  'end': 9},
 {'morpheme': '에서',
  'pos': 'JKB',
  'pos_desc': '부사격 조사',
  'matched_items': [],
  'start': 9,
  'len': 2,
  'end': 11},
 {'morpheme': '나',
  'pos': 'JX',
  'pos_desc': '보조사',
  'mat

In [9]:
# 잘 틀리는 것 예시 잘 못하는

text = """
인생은 짧고 예술은 길다.
사과는 먹어도 배는 먹지 마라.
"""

final_output = main(text)
final_output

형태소 '인생' (위치: 1, 길이: 2, 품사: NNG - 일반 명사)
형태소 '은' (위치: 3, 길이: 1, 품사: JX - 보조사)는 문법 항목 번호 4에 해당합니다: 강조의 뜻을 나타내는 보조사
형태소 '짧' (위치: 5, 길이: 1, 품사: VA - 형용사)
형태소 '고' (위치: 6, 길이: 1, 품사: EC - 연결 어미)
형태소 '예술' (위치: 8, 길이: 2, 품사: NNG - 일반 명사)
형태소 '은' (위치: 10, 길이: 1, 품사: JX - 보조사)는 문법 항목 번호 4에 해당합니다: 강조의 뜻을 나타내는 보조사
형태소 '길' (위치: 12, 길이: 1, 품사: VA - 형용사)
형태소 '다' (위치: 13, 길이: 1, 품사: EF - 종결 어미)는 문법 항목 번호 14에 해당합니다: 해라할 자리에 쓰여, 어떤 사건이나 사실, 상태를 서술하는 뜻을 나타내는 종결 어미
형태소 '.' (위치: 14, 길이: 1, 품사: SF - 종결 부호(. ! ?))
형태소 '사과' (위치: 16, 길이: 2, 품사: NNG - 일반 명사)
형태소 '는' (위치: 18, 길이: 1, 품사: JX - 보조사)는 문법 항목 번호 4에 해당합니다: 강조의 뜻을 나타내는 보조사
형태소 '먹' (위치: 20, 길이: 1, 품사: VV - 동사)
형태소 '어도' (위치: 21, 길이: 2, 품사: EC - 연결 어미)
형태소 '배' (위치: 24, 길이: 1, 품사: NNG - 일반 명사)
형태소 '는' (위치: 25, 길이: 1, 품사: JX - 보조사)는 문법 항목 번호 4에 해당합니다: 강조의 뜻을 나타내는 보조사
형태소 '먹' (위치: 27, 길이: 1, 품사: VV - 동사)
형태소 '지' (위치: 28, 길이: 1, 품사: EC - 연결 어미)
형태소 '말' (위치: 30, 길이: 1, 품사: VX - 보조 용언)
형태소 '어라' (위치: 31, 길이: 1, 품사: EF - 종결 어미)
형태소 '.' (위치: 32, 길이: 1, 품사: SF - 종결

[{'morpheme': '인생',
  'pos': 'NNG',
  'pos_desc': '일반 명사',
  'matched_items': [],
  'start': 1,
  'len': 2,
  'end': 3},
 {'morpheme': '은',
  'pos': 'JX',
  'pos_desc': '보조사',
  'matched_items': [{'번호': 4,
    '형태': '은/는',
    '품사': 'JX',
    '의미': '강조의 뜻을 나타내는 보조사'}],
  'start': 3,
  'len': 1,
  'end': 4},
 {'morpheme': '짧',
  'pos': 'VA',
  'pos_desc': '형용사',
  'matched_items': [],
  'start': 5,
  'len': 1,
  'end': 6},
 {'morpheme': '고',
  'pos': 'EC',
  'pos_desc': '연결 어미',
  'matched_items': [],
  'start': 6,
  'len': 1,
  'end': 7},
 {'morpheme': '예술',
  'pos': 'NNG',
  'pos_desc': '일반 명사',
  'matched_items': [],
  'start': 8,
  'len': 2,
  'end': 10},
 {'morpheme': '은',
  'pos': 'JX',
  'pos_desc': '보조사',
  'matched_items': [{'번호': 4,
    '형태': '은/는',
    '품사': 'JX',
    '의미': '강조의 뜻을 나타내는 보조사'}],
  'start': 10,
  'len': 1,
  'end': 11},
 {'morpheme': '길',
  'pos': 'VA',
  'pos_desc': '형용사',
  'matched_items': [],
  'start': 12,
  'len': 1,
  'end': 13},
 {'morpheme': '다',
  'po

In [10]:
#'matched_items'가 비어있으면 삭제

final_output = [
{'morpheme': item['morpheme']} if not item['matched_items'] else item
    for item in final_output
]
final_output

[{'morpheme': '인생'},
 {'morpheme': '은',
  'pos': 'JX',
  'pos_desc': '보조사',
  'matched_items': [{'번호': 4,
    '형태': '은/는',
    '품사': 'JX',
    '의미': '강조의 뜻을 나타내는 보조사'}],
  'start': 3,
  'len': 1,
  'end': 4},
 {'morpheme': '짧'},
 {'morpheme': '고'},
 {'morpheme': '예술'},
 {'morpheme': '은',
  'pos': 'JX',
  'pos_desc': '보조사',
  'matched_items': [{'번호': 4,
    '형태': '은/는',
    '품사': 'JX',
    '의미': '강조의 뜻을 나타내는 보조사'}],
  'start': 10,
  'len': 1,
  'end': 11},
 {'morpheme': '길'},
 {'morpheme': '다',
  'pos': 'EF',
  'pos_desc': '종결 어미',
  'matched_items': [{'번호': 14,
    '형태': '다',
    '품사': 'EF',
    '의미': '해라할 자리에 쓰여, 어떤 사건이나 사실, 상태를 서술하는 뜻을 나타내는 종결 어미'}],
  'start': 13,
  'len': 1,
  'end': 14},
 {'morpheme': '.'},
 {'morpheme': '사과'},
 {'morpheme': '는',
  'pos': 'JX',
  'pos_desc': '보조사',
  'matched_items': [{'번호': 4,
    '형태': '은/는',
    '품사': 'JX',
    '의미': '강조의 뜻을 나타내는 보조사'}],
  'start': 18,
  'len': 1,
  'end': 19},
 {'morpheme': '먹'},
 {'morpheme': '어도'},
 {'morpheme': '배'},
 {'mor

In [11]:
final_output

[{'morpheme': '인생'},
 {'morpheme': '은',
  'pos': 'JX',
  'pos_desc': '보조사',
  'matched_items': [{'번호': 4,
    '형태': '은/는',
    '품사': 'JX',
    '의미': '강조의 뜻을 나타내는 보조사'}],
  'start': 3,
  'len': 1,
  'end': 4},
 {'morpheme': '짧'},
 {'morpheme': '고'},
 {'morpheme': '예술'},
 {'morpheme': '은',
  'pos': 'JX',
  'pos_desc': '보조사',
  'matched_items': [{'번호': 4,
    '형태': '은/는',
    '품사': 'JX',
    '의미': '강조의 뜻을 나타내는 보조사'}],
  'start': 10,
  'len': 1,
  'end': 11},
 {'morpheme': '길'},
 {'morpheme': '다',
  'pos': 'EF',
  'pos_desc': '종결 어미',
  'matched_items': [{'번호': 14,
    '형태': '다',
    '품사': 'EF',
    '의미': '해라할 자리에 쓰여, 어떤 사건이나 사실, 상태를 서술하는 뜻을 나타내는 종결 어미'}],
  'start': 13,
  'len': 1,
  'end': 14},
 {'morpheme': '.'},
 {'morpheme': '사과'},
 {'morpheme': '는',
  'pos': 'JX',
  'pos_desc': '보조사',
  'matched_items': [{'번호': 4,
    '형태': '은/는',
    '품사': 'JX',
    '의미': '강조의 뜻을 나타내는 보조사'}],
  'start': 18,
  'len': 1,
  'end': 19},
 {'morpheme': '먹'},
 {'morpheme': '어도'},
 {'morpheme': '배'},
 {'mor

In [2]:
과제는 상히를 너무 힘들게 한다.
다른 사람은 모르겠고 나는 예쁘다.
그는 아침에 커피를 마시고, 나는 마시지 않았다.
그는 아침에 커피를 마시고, 나는 그와 함께 신문을 읽었다.
그녀가이일을얼마나오래준비하었는지에대하어서는그누구도모른다
그 식당은 음식이 맛있고, 분위기도 좋다
인생은 짧고 예술은 길며, 사람은 누구나 실수를 한다는 것을 깨닫는 것은 중요한 일이다.

NameError: name 'kiwi' is not defined

In [11]:
# 함수 내에서 동기적으로 호출
def chatgpt(text, matched_items, pna_token, now_token):
    response = openai.ChatCompletion.create(
        model="gpt-4", #gpt-4 ,gpt-4o-mini , gpt-4o, gpt-3.5-turbo
        messages=[{
            "role": "user",
            "content": f"""
            sentence: {text}
            pna_token : {pna_token}
            now_token : {now_token}
            matched_items: {matched_items}

            현재 나는 한국어 문장 태깅에서 판단하기 어려운 부분을 너에게 맡기려고 해.
            sentence는 전체 문장이고, pna_token의 가운데인 now_token의 태깅을 판단해야해.
            matched_items는 현재 태깅의 어려운 부분이야. 이 부분이 2가지 이상의 태깅이 되어있어.
            matched_items의 번호가 몇번이 정답일지 판단해서 숫자로 정답만 알려줘
            matched_items의 번호가 0번인 경우 nlp모델이 분류한 결과야 이게 맞을 수 있는데 나머지 번호가 틀렸다고 판단될 경우 0번으로 알려줘
            그렇지 않은 경우 반드시 matched_items의 번호 중에서의 숫자만 알려줘.
            반드시 설명하지말고 숫자만 알려줘.
            """
        }],
        stream=False,  # 스트리밍 비활성화
    )

    # 응답 처리
    full_text = str(response['choices'][0]['message']['content'])
    print("====GPT answer====", full_text)

    return full_text

In [12]:
final_output

[{'morpheme': '인생', 'pos': 'NNG', 'pos_desc': '일반 명사', 'matched_items': []},
 {'morpheme': '은',
  'pos': 'JX',
  'pos_desc': '보조사',
  'matched_items': [{'번호': 4,
    '형태': '은/는',
    '품사': 'JX',
    '의미': '강조의 뜻을 나타내는 보조사'}]},
 {'morpheme': '짧', 'pos': 'VA', 'pos_desc': '형용사', 'matched_items': []},
 {'morpheme': '고', 'pos': 'EC', 'pos_desc': '연결 어미', 'matched_items': []},
 {'morpheme': '예술', 'pos': 'NNG', 'pos_desc': '일반 명사', 'matched_items': []},
 {'morpheme': '은',
  'pos': 'JX',
  'pos_desc': '보조사',
  'matched_items': [{'번호': 4,
    '형태': '은/는',
    '품사': 'JX',
    '의미': '강조의 뜻을 나타내는 보조사'}]},
 {'morpheme': '길', 'pos': 'VA', 'pos_desc': '형용사', 'matched_items': []},
 {'morpheme': '다',
  'pos': 'EF',
  'pos_desc': '종결 어미',
  'matched_items': [{'번호': 14,
    '형태': '다',
    '품사': 'EF',
    '의미': '해라할 자리에 쓰여, 어떤 사건이나 사실, 상태를 서술하는 뜻을 나타내는 종결 어미'}]},
 {'morpheme': '.',
  'pos': 'SF',
  'pos_desc': '종결 부호(. ! ?)',
  'matched_items': []},
 {'morpheme': '사과', 'pos': 'NNG', 'pos_desc': '일반 명사', 

In [13]:
final_output

[{'morpheme': '인생', 'pos': 'NNG', 'pos_desc': '일반 명사', 'matched_items': []},
 {'morpheme': '은',
  'pos': 'JX',
  'pos_desc': '보조사',
  'matched_items': [{'번호': 4,
    '형태': '은/는',
    '품사': 'JX',
    '의미': '강조의 뜻을 나타내는 보조사'}]},
 {'morpheme': '짧', 'pos': 'VA', 'pos_desc': '형용사', 'matched_items': []},
 {'morpheme': '고', 'pos': 'EC', 'pos_desc': '연결 어미', 'matched_items': []},
 {'morpheme': '예술', 'pos': 'NNG', 'pos_desc': '일반 명사', 'matched_items': []},
 {'morpheme': '은',
  'pos': 'JX',
  'pos_desc': '보조사',
  'matched_items': [{'번호': 4,
    '형태': '은/는',
    '품사': 'JX',
    '의미': '강조의 뜻을 나타내는 보조사'}]},
 {'morpheme': '길', 'pos': 'VA', 'pos_desc': '형용사', 'matched_items': []},
 {'morpheme': '다',
  'pos': 'EF',
  'pos_desc': '종결 어미',
  'matched_items': [{'번호': 14,
    '형태': '다',
    '품사': 'EF',
    '의미': '해라할 자리에 쓰여, 어떤 사건이나 사실, 상태를 서술하는 뜻을 나타내는 종결 어미'}]},
 {'morpheme': '.',
  'pos': 'SF',
  'pos_desc': '종결 부호(. ! ?)',
  'matched_items': []},
 {'morpheme': '사과', 'pos': 'NNG', 'pos_desc': '일반 명사', 

In [14]:
{
    "{\"sentence_text\": \"돈없으면편의점에서나뭐사어먹자.\", \"matched_items\": [{\"번호\": 21, \"형태\": \"(이)나/이나/나\", \"품사\": \"JX\", \"의미\": \"둘 이상의 사물을 같은 자격으로 이어 주는 접속 조사\"}, {\"번호\": 22, \"형태\": \"(이)나/이나/나\", \"품사\": \"JX\", \"의미\": \"수량이 크거나 많음, 혹은 정도가 높음을 강조하는 보조사\"}, {\"번호\": 23, \"형태\": \"(이)나/나/이나\", \"품사\": \"JX\", \"의미\": \"마음에 차지 않는 선택, 또는 최소한 허용되어야 할 선택이라는 뜻을 나타내는 보조사\"}], \"pna_token\": \"에서나뭐\", \"now_token\": \"나\"}": 23,
    "{\"sentence_text\": \"이거나저거나상관없어.\", \"matched_items\": [{\"번호\": 21, \"형태\": \"(이)나/이나/나\", \"품사\": \"JX\", \"의미\": \"둘 이상의 사물을 같은 자격으로 이어 주는 접속 조사\"}, {\"번호\": 22, \"형태\": \"(이)나/이나/나\", \"품사\": \"JX\", \"의미\": \"수량이 크거나 많음, 혹은 정도가 높음을 강조하는 보조사\"}, {\"번호\": 23, \"형태\": \"(이)나/나/이나\", \"품사\": \"JX\", \"의미\": \"마음에 차지 않는 선택, 또는 최소한 허용되어야 할 선택이라는 뜻을 나타내는 보조사\"}], \"pna_token\": \"저거나상관없\", \"now_token\": \"나\"}": 23,
    "{\"sentence_text\": \"지금커피나한잔할까?\", \"matched_items\": [{\"번호\": 21, \"형태\": \"(이)나/이나/나\", \"품사\": \"JX\", \"의미\": \"둘 이상의 사물을 같은 자격으로 이어 주는 접속 조사\"}, {\"번호\": 22, \"형태\": \"(이)나/이나/나\", \"품사\": \"JX\", \"의미\": \"수량이 크거나 많음, 혹은 정도가 높음을 강조하는 보조사\"}, {\"번호\": 23, \"형태\": \"(이)나/나/이나\", \"품사\": \"JX\", \"의미\": \"마음에 차지 않는 선택, 또는 최소한 허용되어야 할 선택이라는 뜻을 나타내는 보조사\"}], \"pna_token\": \"커피나한\", \"now_token\": \"나\"}": 21,
    "{\"sentence_text\": \"광화문에는이순신장군동상이있습니다.\", \"matched_items\": [{\"번호\": 10, \"형태\": \"에\", \"품사\": \"JKB\", \"의미\": \"앞말이 처소의 부사어임을 나타내는 격 조사 또는 진행 방향의 부사어임을 나타내는 격 조사\"}, {\"번호\": 11, \"형태\": \"에\", \"품사\": \"JKB\", \"의미\": \"앞말이 시간의 부사어임을 나타내는 격 조사\"}], \"pna_token\": \"광화문에는\", \"now_token\": \"에\"}": 10
}

{'{"sentence_text": "돈없으면편의점에서나뭐사어먹자.", "matched_items": [{"번호": 21, "형태": "(이)나/이나/나", "품사": "JX", "의미": "둘 이상의 사물을 같은 자격으로 이어 주는 접속 조사"}, {"번호": 22, "형태": "(이)나/이나/나", "품사": "JX", "의미": "수량이 크거나 많음, 혹은 정도가 높음을 강조하는 보조사"}, {"번호": 23, "형태": "(이)나/나/이나", "품사": "JX", "의미": "마음에 차지 않는 선택, 또는 최소한 허용되어야 할 선택이라는 뜻을 나타내는 보조사"}], "pna_token": "에서나뭐", "now_token": "나"}': 23,
 '{"sentence_text": "이거나저거나상관없어.", "matched_items": [{"번호": 21, "형태": "(이)나/이나/나", "품사": "JX", "의미": "둘 이상의 사물을 같은 자격으로 이어 주는 접속 조사"}, {"번호": 22, "형태": "(이)나/이나/나", "품사": "JX", "의미": "수량이 크거나 많음, 혹은 정도가 높음을 강조하는 보조사"}, {"번호": 23, "형태": "(이)나/나/이나", "품사": "JX", "의미": "마음에 차지 않는 선택, 또는 최소한 허용되어야 할 선택이라는 뜻을 나타내는 보조사"}], "pna_token": "저거나상관없", "now_token": "나"}': 23,
 '{"sentence_text": "지금커피나한잔할까?", "matched_items": [{"번호": 21, "형태": "(이)나/이나/나", "품사": "JX", "의미": "둘 이상의 사물을 같은 자격으로 이어 주는 접속 조사"}, {"번호": 22, "형태": "(이)나/이나/나", "품사": "JX", "의미": "수량이 크거나 많음, 혹은 정도가 높음을 강조하는 보조사"}, {"번호": 23, "형태": "(이)나/나/이나", "품사": "JX", "의미"

In [19]:
# 문제 


# 추가 해결 방안 설정 
돈 없으면 편의점에서나 뭐 사 먹자. => 21,22,23 전부 태깅 => 23으로 태깅되어야
이거나 저거나 상관없어. => 21,22,23 전부 태깅 => 21로 태깅되어야 함
지금 커피나 한 잔 할까? => 21,22,23 전부 태깅 => 22번 태깅되어야 함.

# 기타 
건물 안 귀신이 나를 잡아먹는다 => (25번 태깅) 우물 안 귀신 은 안됨 우물 안의 귀신은 됨


# 재확인 필요
마음에 들어 => "에"가 장소로 나오는데 이거 로직 수정해서 만들어야 할 것으로 보임. 대공사.?=> 우선 else pass로 로직 변경 => gpt가 최종판단 예외인 경우가 많음. 세분화를 하던지
건물 안 귀신이 나를 잡아먹는다 => (25번 태깅) 우물 안 귀신 은 안됨 우물 안의 귀신은 됨


# 완료 
개 보고 나와서 춤이나 추라고 그래. => 이 문장의 "보고"는 연결 어미 + 동사로 나옴 (한번 보고 두번 보고 자꾸만 보고 싶네) 이런 경우 어떻게 하지.. =>문제 덮기? 보고가 생각보다 잘 안잡힘. => 문장을 "보고"가 들어갈 경우 앞의 명사와 띄어씌기를 하면 안됨(ex 엄마 보고 이걸 하라고 했어.)
숟가락을 들 수가 없어요.=> "수가"에서 "가"태깅이 잘못 됨. => "가"가 주격 조사가 맞다고 함.
너무 피곤해서 더는 못 앉아있겠습니다. 누워서 일하게 해주세요. => 부사들이 사이에 있다. 어떻게 해결해야 할까..이건 pass 그 어뜬 것도 없음 추후 gpt로 실험해보기
다만 다음부터는 나한테 뭐가 필요한지 꼭 물어보고 사주길 바랄게. => "는"은 2,3,4에 다 걸림..이건 pass 그 어뜬 것도 없음 추후 gpt로 실험해보기

내가 오후에 모이라고 했는데 어디에 가 있었던 거야? => 오후에의  "에"는 처소가 아니라 시간 => time_nouns에 오후 추가. => 추가 필요 문장들 추가 [11번]


SyntaxError: invalid decimal literal (1795767187.py, line 5)

In [20]:
text = """
개 보고 나와서 춤이나 추라고 그래
"""

final_output = main(text)

{'번호': 16, '형태': '보고', '품사': 'JKB', '의미': '어떤 행동이 미치는 대상임을 나타내는 격 조사'}
형태소 '개' (위치: 1, 길이: 1, 품사: NNG - 일반 명사)
형태소 '보고' (위치: 3, 길이: 2, 품사: JKB - 어떤 행동이 미치는 대상임을 나타내는 격 조사)는 문법 항목 번호 16에 해당합니다: 어떤 행동이 미치는 대상임을 나타내는 격 조사
형태소 '나오' (위치: 6, 길이: 2, 품사: VV - 동사)
형태소 '어서' (위치: 7, 길이: 2, 품사: EC - 연결 어미)
형태소 '춤' (위치: 10, 길이: 1, 품사: NNG - 일반 명사)
형태소 '이나' (위치: 11, 길이: 2, 품사: JX - 보조사)는 문법 항목 번호 23에 해당합니다: 마음에 차지 않는 선택, 또는 최소한 허용되어야 할 선택이라는 뜻을 나타내는 보조사
형태소 '추' (위치: 14, 길이: 1, 품사: VV - 동사)
형태소 '라고' (위치: 15, 길이: 2, 품사: EC - 연결 어미)
형태소 '그래' (위치: 18, 길이: 2, 품사: IC - 감탄사)


In [17]:

# 잘 틀리는 것 예시 잘 못하는

text = """
그 학생 보고 보고서를 꼭 발표하라고 하셨다.
이것은 나의 것이다.
오늘은 금요일이다.
운동이나 시작해 볼까 생각 중이야.
이거 드세요. **
이것은 나의 것이다.
오늘은 금요일이다.
오늘 아침에는 안 추웠어.

할아버지께서는 건강이 좋으십니다.
사과나 배 중에 어떤 걸 좋아해?
축구나 농구를 할 수 있어.
늦었으니 택배는 내일이나 받아야겠다.
"""


final_output = main(text)

형태소 '그' (품사: IC - 감탄사)
형태소 '학생' (품사: NNG - 일반 명사)
형태소 '보고' (품사: JKB - 어떤 행동이 미치는 대상임을 나타내는 격 조사)는 문법 항목 번호 16에 해당합니다: 어떤 행동이 미치는 대상임을 나타내는 격 조사
형태소 '보고서' (품사: NNG - 일반 명사)
형태소 '를' (품사: JKO - 목적격 조사)
형태소 '꼭' (품사: MAG - 일반 부사)
형태소 '발표' (품사: NNG - 일반 명사)
형태소 '하' (품사: XSV - 동사 파생 접미사)
형태소 '라고' (품사: EC - 연결 어미)
형태소 '하' (품사: VV - 동사)
형태소 '시' (품사: EP - 선어말 어미)는 문법 항목 번호 17에 해당합니다: 높임의 뜻을 더하는 선어말 어미
형태소 '었' (품사: EP - 선어말 어미)
형태소 '다' (품사: EF - 종결 어미)는 문법 항목 번호 14에 해당합니다: 해라할 자리에 쓰여, 어떤 사건이나 사실, 상태를 서술하는 뜻을 나타내는 종결 어미
형태소 '.' (품사: SF - 종결 부호(. ! ?))
형태소 '이것' (품사: NP - 대명사)
형태소 '은' (품사: JX - 보조사)는 문법 항목 번호 2에 해당합니다: 문장 속에서 어떤 대상이 화제임을 나타내는 보조사
형태소 '나' (품사: NP - 대명사)
형태소 '의' (품사: JKG - 관형격 조사)
형태소 '것' (품사: NNB - 의존 명사)
형태소 '이' (품사: VCP - 긍정 지시사(이다))
형태소 '다' (품사: EF - 종결 어미)는 문법 항목 번호 14에 해당합니다: 해라할 자리에 쓰여, 어떤 사건이나 사실, 상태를 서술하는 뜻을 나타내는 종결 어미
형태소 '.' (품사: SF - 종결 부호(. ! ?))
형태소 '오늘' (품사: NNG - 일반 명사)
형태소 '은' (품사: JX - 보조사)는 문법 항목 번호 2에 해당합니다: 문장 속에서 어떤 대상이 화제임을 나타내는 보조사
형태소 '금요일' (품사: NNG - 일반 명사)
형

In [18]:
# 1번 예시
text = """
밥을 먹은 사람이 바로 이가 아니다.
나무가 많다는데 산이 없다는 게 말이 되냐?
그 사람이 누군지 모르는 사람이 많다.
너는 일을 끝내기 전까지 걱정이 많겠구나.
시간만 있으면 더 공부할 텐데, 시간이 너무 없다.
나에게 시간이 많지 않다.
물이 차가워질 때까지 그냥 둬라.
내일 모레가 중요해.
산책로가 길다.
우리 집 개가 저쪽으로 간다.
비가 오면 집안일이 산더미처럼 쌓인다.
커피가 떨어졌네, 가서 사올래?
내 친구가 너한테 줄 게 있다.
그 이야기가 사실이라면 문제가 크다.
날씨가 점점 더워지면 바깥 활동이 어려워진다.
동생이 자꾸만 날 따라온다.
문이 열려 있는데 아무도 안 들어간다.
밥이 맛있다고 해서 먹어봤다.
그 사람이 한 말이 맞다면 나는 곤란해진다.
집이 크다면 어쩌겠어?
바람이 잠잠해지고 있다가 갑자기 다시 세졌다.
"""
final_output = main(text)

형태소 '밥' (품사: NNG - 일반 명사)
형태소 '을' (품사: JKO - 목적격 조사)
형태소 '먹' (품사: VV - 동사)
형태소 '은' (품사: ETM - 관형형 전성 어미)는 문법 항목 번호 5에 해당합니다: 앞말이 관형어 구실을 하게 하고 동작이 과거에 이루어졌음을 나타내는 어미
형태소 '사람' (품사: NNG - 일반 명사)
형태소 '이' (품사: JKS - 주격 조사)는 문법 항목 번호 1에 해당합니다: 어떤 상태를 보이는 대상이나 일정한 상태나 상황을 겪는 경험주 또는 일정한 동작의 주체임을 나타내는 격 조사
형태소 '바로' (품사: MAG - 일반 부사)
형태소 '이' (품사: MM - 관형사)는 문법 항목 번호 7에 해당합니다: 말하는 이에게 가까이 있거나 말하는 이가 생각하고 있는 대상을 가리킬 때 쓰는 말
형태소 '가' (품사: JKC - 보격 조사)
형태소 '아니' (품사: VCN - 부정 지시사(아니다))
형태소 '다' (품사: EF - 종결 어미)는 문법 항목 번호 14에 해당합니다: 해라할 자리에 쓰여, 어떤 사건이나 사실, 상태를 서술하는 뜻을 나타내는 종결 어미
형태소 '.' (품사: SF - 종결 부호(. ! ?))
형태소 '나무' (품사: NNG - 일반 명사)
형태소 '가' (품사: JKS - 주격 조사)는 문법 항목 번호 1에 해당합니다: 어떤 상태를 보이는 대상이나 일정한 상태나 상황을 겪는 경험주 또는 일정한 동작의 주체임을 나타내는 격 조사
형태소 '많' (품사: VA - 형용사)
형태소 '다는' (품사: ETM - 관형형 전성 어미)
형태소 '데' (품사: NNB - 의존 명사)
형태소 '산' (품사: NNG - 일반 명사)
형태소 '이' (품사: JKS - 주격 조사)는 문법 항목 번호 1에 해당합니다: 어떤 상태를 보이는 대상이나 일정한 상태나 상황을 겪는 경험주 또는 일정한 동작의 주체임을 나타내는 격 조사
형태소 '없' (품사: VA - 형용사)
형태소 '다는' (품사: ETM - 관형형 전성 어미)

In [22]:
"""오늘은 금요일이다.
나는 학생이다.
집에서 공부하는 것은 중요하다.
내일이 중요한 날은 맞다.
내가 좋아하는 영화는 바로 이거야.
시간이 없다는 것은 말도 안 된다.
할 일은 많지만 시간이 없다.
중요한 것은 네가 무엇을 하느냐이다.
내가 하고 싶은 것은 모두에게 말하는 것이다.
먹고 싶은 것은 다 먹어라.
그 사람이 선생님은 아니다.
문제가 되는 것은 그의 태도이다.
그 사람은 너의 친구는 아닐 거야.
우리가 할 일은 이게 전부가 아니다.
중요한 문제는 지금부터 해결하겠다.
내가 원하는 것은 좋은 직장이다.
내가 하고 있는 일은 꽤 재미있다.
그는 항상 바쁜 사람은 아니다.
네가 좋아하는 것은 무엇이니?
내가 기억하는 사람은 너뿐이다.
할 수 있는 것은 다 해봤다.
말하고 싶은 것은 더 이상 없다."""


# 2번 예시
text = """오늘은 날씨가 참 좋다.
나는 내일 여행을 갈 거야.
책은 이미 다 읽었다.
이것은 정말 중요한 문제다.
그 사람은 내 친구야.
컴퓨터는 고장 났어.
내일은 학교에 안 가도 된다.
저 영화는 너무 재미있어.
이 집은 너무 크다.
이번 시험은 어렵다."""
final_output = main(text)

형태소 '오늘' (품사: NNG - 일반 명사)
형태소 '은' (품사: JX - 보조사)는 문법 항목 번호 2에 해당합니다: 문장 속에서 어떤 대상이 화제임을 나타내는 보조사
형태소 '날씨' (품사: NNG - 일반 명사)
형태소 '가' (품사: JKS - 주격 조사)는 문법 항목 번호 1에 해당합니다: 어떤 상태를 보이는 대상이나 일정한 상태나 상황을 겪는 경험주 또는 일정한 동작의 주체임을 나타내는 격 조사
형태소 '참' (품사: MAG - 일반 부사)
형태소 '좋' (품사: VA - 형용사)
형태소 '다' (품사: EF - 종결 어미)는 문법 항목 번호 14에 해당합니다: 해라할 자리에 쓰여, 어떤 사건이나 사실, 상태를 서술하는 뜻을 나타내는 종결 어미
형태소 '.' (품사: SF - 종결 부호(. ! ?))
형태소 '나' (품사: NP - 대명사)
형태소 '는' (품사: JX - 보조사)는 문법 항목 번호 2에 해당합니다: 문장 속에서 어떤 대상이 화제임을 나타내는 보조사
형태소 '내일' (품사: NNG - 일반 명사)
형태소 '여행' (품사: NNG - 일반 명사)
형태소 '을' (품사: JKO - 목적격 조사)
형태소 '가' (품사: VV - 동사)
형태소 'ᆯ' (품사: ETM - 관형형 전성 어미)
형태소 '거' (품사: NNB - 의존 명사)
형태소 '이' (품사: VCP - 긍정 지시사(이다))
형태소 '야' (품사: EF - 종결 어미)
형태소 '.' (품사: SF - 종결 부호(. ! ?))
형태소 '책' (품사: NNG - 일반 명사)
형태소 '은' (품사: JX - 보조사)는 문법 항목 번호 2에 해당합니다: 문장 속에서 어떤 대상이 화제임을 나타내는 보조사
형태소 '이미' (품사: MAG - 일반 부사)
형태소 '다' (품사: MAG - 일반 부사)는 문법 항목 번호 15에 해당합니다: 남거나 빠진 것이 없이 모두
형태소 '읽' (품사: VV - 동사)
형태소 '었' (품사: EP - 선어말 어미)
형태소 '다' (품사: EF - 

In [23]:

"""
이 차는 빠르고, 저 차는 느리다."""

# 3번 예시 
text = """
인생은 짧고 예술은 길다.
사과는 먹어도 배는 먹지 마라.
나는 피자를 좋아하지만 동생은 파스타를 좋아한다.
사과는 맛있지만 배는 그렇지 않다.
나는 일을 잘하지만 친구는 그렇지 않다.
사람은 변하지만 세상은 그대로다.
나는 공부를 열심히 하는데, 동생은 그렇지 않다.
오늘은 비가 오고, 내일은 맑을 것이다.
그는 말을 많이 하지만 나는 말이 없다.
아버지는 신문을 읽고, 어머니는 책을 읽는다.
나는 커피를 좋아하는데, 동생은 차를 좋아한다.
겨울은 춥고, 여름은 덥다.
고양이는 얌전하지만 강아지는 활발하다.
나는 운동을 좋아하는데 친구는 게임을 좋아한다.
학교는 갔지만 공부는 하지 않았다.
그는 운동을 잘하지만 나는 그렇지 않다.
그녀는 요리를 잘하지만 나는 별로다.
이번 주는 바빴고, 다음 주는 여유로울 것이다.
차는 마셨지만 밥은 먹지 않았다.
그 사람은 열심히 일하지만, 나는 그만큼 하지 못했다.
동생은 숙제를 다 끝냈지만 나는 아직 남았다.
이 옷은 편하지만 저 옷은 불편하다.
그는 책을 읽고, 나는 영화를 본다.
오늘은 축구를 했지만 내일은 농구를 할 것이다.
오늘은 날씨가 좋지만, 내일은 비가 올지도 몰라.
나는 책을 읽고, 친구는 영화를 본다.
저 사람은 선생님이고, 이 사람은 학생이다.
그 영화는 재미있었지만, 결말은 아쉬웠다.
이 과일은 사과고, 저 과일은 배야.
여름에는 더워서, 겨울에는 추워서 힘들다.
그는 키가 크고, 나는 키가 작다.
사과는 먹었는데, 배는 남겼다.
이 길은 짧지만, 저 길은 길다.
이번 주는 바쁘고, 다음 주는 한가할 거야.
이 사람은 축구를 좋아하고, 저 사람은 야구를 좋아해.
오늘은 쉬고, 내일은 일할 거야.
그 가게는 가격이 싸지만, 품질은 좋지 않다.
이 문제는 어렵고, 저 문제는 쉽다.
나는 커피를 좋아하지만, 동생은 차를 좋아해.
그 식당은 음식이 맛있고, 분위기도 좋다.
학교는 갔지만, 공부는 하지 않았다.
오늘은 아침을 먹었고, 점심은 건너뛰었다.
나는 그 사람을 좋아하지만, 그는 나를 싫어한다.
이 차는 빠르고, 저 차는 느리다.
"""
final_output = main(text)

모든 조건 만족
모든 조건 만족
형태소 '인생' (품사: NNG - 일반 명사)
형태소 '은' (품사: JX - 보조사)는 문법 항목 번호 3에 해당합니다: 어떤 대상이 다른 것과 대조됨을 나타내는 보조사
형태소 '짧' (품사: VA - 형용사)
형태소 '고' (품사: EC - 연결 어미)
형태소 '예술' (품사: NNG - 일반 명사)
형태소 '은' (품사: JX - 보조사)는 문법 항목 번호 3에 해당합니다: 어떤 대상이 다른 것과 대조됨을 나타내는 보조사
형태소 '길' (품사: VA - 형용사)
형태소 '다' (품사: EF - 종결 어미)는 문법 항목 번호 14에 해당합니다: 해라할 자리에 쓰여, 어떤 사건이나 사실, 상태를 서술하는 뜻을 나타내는 종결 어미
모든 조건 만족
모든 조건 만족
형태소 '.' (품사: SF - 종결 부호(. ! ?))
형태소 '사과' (품사: NNG - 일반 명사)
형태소 '는' (품사: JX - 보조사)는 문법 항목 번호 3에 해당합니다: 어떤 대상이 다른 것과 대조됨을 나타내는 보조사
형태소 '먹' (품사: VV - 동사)
형태소 '어도' (품사: EC - 연결 어미)
형태소 '배' (품사: NNG - 일반 명사)
형태소 '는' (품사: JX - 보조사)는 문법 항목 번호 3에 해당합니다: 어떤 대상이 다른 것과 대조됨을 나타내는 보조사
형태소 '먹' (품사: VV - 동사)
형태소 '지' (품사: EC - 연결 어미)
형태소 '말' (품사: VX - 보조 용언)
형태소 '어라' (품사: EF - 종결 어미)
형태소 '.' (품사: SF - 종결 부호(. ! ?))
형태소 '나' (품사: NP - 대명사)
형태소 '는' (품사: JX - 보조사)는 문법 항목 번호 2에 해당합니다: 문장 속에서 어떤 대상이 화제임을 나타내는 보조사
형태소 '피자' (품사: NNG - 일반 명사)
형태소 '를' (품사: JKO - 목적격 조사)
형태소 '좋아하' (품사: VV - 동사)
형태소 '지만' (품사: EC - 연결 어미)
형태소 '

In [24]:
# 4번 예시 =>강조의 뜻을 나타내는 보조사


text = """
너에게도 잘못은 있다.
아무리 바쁘더라도 식사는 해야지.

공부는 열심히 해야 한다.
시간은 누구에게나 공평하다.
이번 프로젝트는 성공해야 한다.
밥은 먹어야 한다.
너의 책임은 확실하다.
이 문제는 반드시 풀어야 한다.
기회는 다시 오지 않을 것이다.
식사는 했니? 시간이 늦었으니 저녁은 꼭 먹어.
이 일은 내가 할 수 있다.
"""
final_output = main(text)

형태소 '너' (품사: NP - 대명사)
형태소 '에게' (품사: JKB - 부사격 조사)
형태소 '도' (품사: JX - 보조사)는 문법 항목 번호 12에 해당합니다: 이미 어떤 것이 포함되고 그 위에 더함의 뜻을 나타내는 보조사
형태소 '잘못' (품사: MAG - 일반 부사)
형태소 '은' (품사: JX - 보조사)는 문법 항목 번호 4에 해당합니다: 강조의 뜻을 나타내는 보조사
형태소 '있' (품사: VV - 동사)
형태소 '다' (품사: EF - 종결 어미)는 문법 항목 번호 14에 해당합니다: 해라할 자리에 쓰여, 어떤 사건이나 사실, 상태를 서술하는 뜻을 나타내는 종결 어미
형태소 '.' (품사: SF - 종결 부호(. ! ?))
형태소 '아무리' (품사: MAG - 일반 부사)
형태소 '바쁘' (품사: VA - 형용사)
형태소 '더라도' (품사: EC - 연결 어미)
형태소 '식사' (품사: NNG - 일반 명사)
형태소 '는' (품사: JX - 보조사)는 문법 항목 번호 2에 해당합니다: 문장 속에서 어떤 대상이 화제임을 나타내는 보조사
형태소 '하' (품사: VV - 동사)
형태소 '어야지' (품사: EF - 종결 어미)
형태소 '.' (품사: SF - 종결 부호(. ! ?))
형태소 '공부' (품사: NNG - 일반 명사)
형태소 '는' (품사: JX - 보조사)는 문법 항목 번호 2에 해당합니다: 문장 속에서 어떤 대상이 화제임을 나타내는 보조사
형태소 '열심히' (품사: MAG - 일반 부사)
형태소 '하' (품사: VV - 동사)
형태소 '어야' (품사: EC - 연결 어미)
형태소 '하' (품사: VX - 보조 용언)
형태소 'ᆫ다' (품사: EF - 종결 어미)
False 조건 5: JX('이나'/'나') 사이에 N* ,동사 (번호 21번)
JX('이나'/'나') 사이에 N*  (번호 21번)
[{'번호': 0, '형태': '나', '품사': 'JX', '의미': '보조사'}, {'번호': 21, '형태': '(이)나/이나/나', '

In [ ]:
# 5번 예시 
text = """어제 내 팔을 잡은 사람이 바로 저 사람이다.
어제 먹은 음식이 정말 맛있었다.
내가 어렸을 때 먹은 음식이 그리워.
방금 읽은 책이 정말 흥미로웠어.

읽은 책이 많다.
보낸 편지가 도착했다.
만든 음식이 맛있다.
잡은 물고기가 크다.
깨진 유리가 위험하다.
끊어진 끈을 고쳤다.
쌓인 눈이 많다.
잃어버린 열쇠를 찾았다.
찾은 물건이 바로 이거다.
깨끗이 청소한 방이다.
고친 컴퓨터가 작동한다.
그는 이미 끝난 일이다.
불이 꺼진 방에 들어갔다.
찢어진 종이가 흩어졌다.
닫힌 문이 열렸다.
사라진 돈을 찾고 있다.
죽은 나무를 베어버렸다.
끓인 물을 식혀 마셨다.
자른 머리가 마음에 든다.
먹은 음식이 소화가 잘됐다.
"""
final_output = main(text)

In [ ]:
# 6번 예시 
text = """웃는 얼굴.
달리는 차를 멈춰라.

지금 달리는 차를 잡을 수 없을 거야.
웃는 얼굴이 너무 밝아 보여.
지금 듣는 노래가 너무 좋아.
가는 길에 비가 내리기 시작했어.
그는 항상 열심히 일하는 사람이다.

웃는 얼굴이 예쁘다.
달리는 차가 위험하다.
먹는 음식이 건강에 좋다.
자는 고양이가 귀엽다.
가는 길이 험하다.
보는 TV가 재미있다.
뛰는 아이가 있다.
살아있는 동물들이 많다.
웃는 사람이 많다.
먹는 과일이 신선하다.
울고 있는 아이가 있다.
쉬는 날이 좋다.
앉는 자세가 바르다.
쓰는 펜이 고장 났다.
넘어지는 사람이 위험하다.
읽는 책이 재미있다.
만드는 과정이 복잡하다.
사는 집이 크다.
깨는 소리가 났다.
뛰는 운동이 건강에 좋다.
"""
final_output = main(text)

In [ ]:
# 7번 예시 
text = """이 사과가 맛있게 생겼다.
이 사과가 맛있다.
이 책을 한번 읽어 봐.

이 사람을 본 적이 있나요?
이게 무엇인지 알아?
이곳이 내 고향이다.
이것은 내 것이다.
이 책을 다 읽었다.
이제 시작하자.
이대로 가자.
이 집에서 나간다.
이 일은 중요해요.
이 물건이 네 것이니?
"""
final_output = main(text)

In [ ]:
# 8번 예시 
text = """그 책 이리 좀 줘 봐.
그 문제는 이미 해결되었다.

그거 정말 좋아.
그는 항상 있었다.
그 사람.
다시 시작하는 것이 점점 더 많아지고 있습니다.
그러길 바랐어요.
그가 내 친구예요.
그게 뭐야?
그 일은 다 끝났다.
그렇게 하지 마.
그 집에 가자.
그 사람은 정말 좋아해요.
"""
final_output = main(text)

In [ ]:
# 9번 예시 
text = """저 거리에는 항상 사람이 많다.

산이 정말 높다.
저 건물은 오래되었다.
저 사람이 내 친구야.
저 길로 가면 집에 도착할 거야.
저 물건은 어디서 샀어?
저 집은 비싸 보인다.
저 강이 꽤 넓다.
저 나무가 정말 크네.
저 하늘이 참 맑다.
저 식당은 음식이 맛있어.
"""
final_output = main(text)

In [ ]:
# 10번 예시 
text = """나는 시골에 산다.
동생은 방금 집에 갔다.
나는 도서관에 간다.
친구가 집에 머무르고 있어.
우리는 공원에 모였다.
아이가 방에 들어갔다.
"""
final_output = main(text)

In [ ]:
# 11번 예시 
text = """그 시간에 뭐 할 거니?
저번에 뭐 했니?
우리는 저녁 6시에 만날 거야
내일 아침에 출발할 계획이야.
저번 주에 무슨 일이 있었어?
겨울에 눈이 많이 내린다.
그때에 우리는 함께 있었어.
"""
final_output = main(text)

In [ ]:
# 12번 예시 
text = """밥만 먹지 말고 반찬도 먹어라.

나도 거기에 갈 거야.
그는 공부도 잘해.
밥도 먹었어?
내 친구도 함께 갈 거야.
시간 있으면 영화도 볼까?
나도 네 말에 동의해.
그 문제도 해결해야 해.
이번 일도 내가 맡을게.
저 사람도 모임에 참여했어.
이 일도 생각보다 쉬웠어.
너도 나도 우리 모두 그 생각에 동의해.
"""
final_output = main(text)

In [ ]:
# 13번 예시 
text = """하루 종일 잠만 잤더니 머리가 띵했다.

오늘만 바쁘고 내일은 한가해.
그 사람만 알고 있어.
나만 늦게 도착했어.
이 책만 읽으면 돼.
내일만 기다려.
이번만 참아 줘.
아침만 먹고 나갈게.
저녁에만 연락해 줘.
이번 시험만 잘 보면 돼.
이 일만 해결하면 다 끝나.
"""
final_output = main(text)

In [ ]:
# 14번 예시 
text = """물이 맑다.

오늘 날씨가 참 좋다.
그 책은 재미있었다.
이제 다 끝났다.
그 사람은 정말 착하다.
이 일은 내가 다 했다.
오늘 하루도 수고 많았다.
나는 그 이야기를 다 들었다.
모든 것이 순조롭게 흘러갔다.
이 문제는 간단하다.
시험이 드디어 끝났다.
"""
final_output = main(text)

In [ ]:
# 15번 예시 
text = """올 사람은 다 왔다.

사람들이 다 떠났다.
오늘 할 일은 다 끝났다.
내일 다 같이 모여야 해.
이 음식을 다 먹을 수 있을까?
영화가 다 끝나고 나서 이야기해.
우리가 계획한 것이 다 잘 풀렸다.
책을 다 읽고 나서 숙제를 해야 해.
여기 있는 물건을 다 챙겼어?
팀원들이 다 모였어.
이번 문제는 다 해결됐어.
"""
final_output = main(text)

In [19]:
# 16번 예시 
text = """
동생이 형보고 장난감을 사 달라고 했다.
누가 너보고 그 일을 하라고 그러더냐?
"""
final_output = main(text)

{'번호': 16, '형태': '보고', '품사': 'JKB', '의미': '어떤 행동이 미치는 대상임을 나타내는 격 조사'}
형태소 '동생' (품사: NNG - 일반 명사)
형태소 '이' (품사: JKS - 주격 조사)는 문법 항목 번호 1에 해당합니다: 어떤 상태를 보이는 대상이나 일정한 상태나 상황을 겪는 경험주 또는 일정한 동작의 주체임을 나타내는 격 조사
형태소 '형' (품사: NNG - 일반 명사)
형태소 '보고' (품사: JKB - 어떤 행동이 미치는 대상임을 나타내는 격 조사)는 문법 항목 번호 16에 해당합니다: 어떤 행동이 미치는 대상임을 나타내는 격 조사
형태소 '장난감' (품사: NNG - 일반 명사)
형태소 '을' (품사: JKO - 목적격 조사)
형태소 '사' (품사: VV - 동사)
형태소 '어' (품사: EC - 연결 어미)
형태소 '달' (품사: VX - 보조 용언)
형태소 '라고' (품사: EC - 연결 어미)
형태소 '하' (품사: VV - 동사)
형태소 '었' (품사: EP - 선어말 어미)
형태소 '다' (품사: EF - 종결 어미)는 문법 항목 번호 14에 해당합니다: 해라할 자리에 쓰여, 어떤 사건이나 사실, 상태를 서술하는 뜻을 나타내는 종결 어미
형태소 '.' (품사: SF - 종결 부호(. ! ?))
형태소 '누구' (품사: NP - 대명사)
형태소 '가' (품사: JKS - 주격 조사)는 문법 항목 번호 1에 해당합니다: 어떤 상태를 보이는 대상이나 일정한 상태나 상황을 겪는 경험주 또는 일정한 동작의 주체임을 나타내는 격 조사
형태소 '너' (품사: NP - 대명사)
형태소 '보고' (품사: JKB - 부사격 조사)는 문법 항목 번호 16에 해당합니다: 어떤 행동이 미치는 대상임을 나타내는 격 조사
형태소 '그' (품사: MM - 관형사)는 문법 항목 번호 8에 해당합니다: 듣는 이에게 가까이 있거나 듣는 이가 생각하고 있는 대상을 가리킬 때 쓰는 말
형태소 '일' (품사: NNG - 일반 명사)
형태소 '을'

In [33]:
# 16번 예시 
text = """
누가 너보고 그 일을 하라고 그러더냐?

보고서를 작성하다.

그 학생 보고 보고서를 꼭 발표하라고 하셨다.
개 보고 나와서 춤이나 추라고 그래.
엄마 보고 이걸 하라고 했어.
선배 보고 그 사람 좀 도와달라고 부탁했어.
아빠 보고 가게 좀 들러 달라고 했어.
친구 보고 이 문서 좀 검토해 달라고 했어.
누가 너보고 그 일을 하라고 그러더냐?
엄마보고 숙제를 끝내라고 하셨어요.
친구보고 같이 가자고 했어.
선생님보고 설명해 달라고 부탁했어.
동생보고 문 좀 열라고 했어.
사장님보고 결정을 내려 달라고 요청했다.
형보고 도와달라고 부탁했어.
그 사람보고 빨리 오라고 했어.
아빠보고 같이 가자고 했어.
친구보고 먼저 가라고 전했어




그는 나보고 공부하라고 했다.
엄마가 너보고 집에 일찍 들어오래.
친구가 나보고 같이 가자고 했다.
선생님이 그 학생보고 발표하라고 하셨다.
동생이 형보고 장난감을 사 달라고 했다.
누가 너보고 그 일을 하라고 했어?
그 친구는 나보고 먼저 가라고 했다.
동료가 상사보고 프로젝트를 맡아 달라고 했다.
친구가 나보고 도움을 요청했다.
선생님이 너보고 숙제를 다시 하라고 하셨어.
"""
final_output = main(text)

형태소 '그' (품사: IC - 감탄사)
형태소 '학생' (품사: NNG - 일반 명사)
형태소 '보고' (품사: JKB - 어떤 행동이 미치는 대상임을 나타내는 격 조사)는 문법 항목 번호 16에 해당합니다: 어떤 행동이 미치는 대상임을 나타내는 격 조사
형태소 '보고서' (품사: NNG - 일반 명사)
형태소 '를' (품사: JKO - 목적격 조사)
형태소 '꼭' (품사: MAG - 일반 부사)
형태소 '발표' (품사: NNG - 일반 명사)
형태소 '하' (품사: XSV - 동사 파생 접미사)
형태소 '라고' (품사: EC - 연결 어미)
형태소 '하' (품사: VV - 동사)
형태소 '시' (품사: EP - 선어말 어미)는 문법 항목 번호 17에 해당합니다: 높임의 뜻을 더하는 선어말 어미
형태소 '었' (품사: EP - 선어말 어미)
형태소 '다' (품사: EF - 종결 어미)는 문법 항목 번호 14에 해당합니다: 해라할 자리에 쓰여, 어떤 사건이나 사실, 상태를 서술하는 뜻을 나타내는 종결 어미
형태소 '.' (품사: SF - 종결 부호(. ! ?))
형태소 '누구' (품사: NP - 대명사)
형태소 '가' (품사: JKS - 주격 조사)는 문법 항목 번호 1에 해당합니다: 어떤 상태를 보이는 대상이나 일정한 상태나 상황을 겪는 경험주 또는 일정한 동작의 주체임을 나타내는 격 조사
형태소 '너' (품사: NP - 대명사)
형태소 '보고' (품사: JKB - 부사격 조사)는 문법 항목 번호 16에 해당합니다: 어떤 행동이 미치는 대상임을 나타내는 격 조사
형태소 '그' (품사: MM - 관형사)는 문법 항목 번호 8에 해당합니다: 듣는 이에게 가까이 있거나 듣는 이가 생각하고 있는 대상을 가리킬 때 쓰는 말
형태소 '일' (품사: NNG - 일반 명사)
형태소 '을' (품사: JKO - 목적격 조사)
형태소 '하' (품사: VV - 동사)
형태소 '라고' (품사: EC - 연결 어미)
형태소 '그러' (품사: VV - 동사)
형태소 '더냐' (품사: 

In [ ]:
# 17번 예시 
text = """선생님은 키가 크시다.
충무공은 훌륭한 장군이셨다.
이거 드세요.
어머니께서 저녁을 준비하십니다.
할아버지께서는 건강이 좋으십니다.
사장님께서 회의를 주재하십니다.
교수님은 연구에 집중하십니다.
어머님께서 방을 청소하셨어요.
부장님께서 보고서를 검토하실 겁니다.
선생님은 학생들에게 책을 읽어주셨습니다.
아버지께서는 매일 아침 운동을 하십니다.
의사 선생님께서 곧 오실 예정입니다.
할머니께서는 요리를 잘하십니다.
사장님께서 질문에 답변하셨습니다.
교장선생님은 학교 행사를 주관하십니다.
어머님께서 생일 케이크를 만드셨어요.
선배님께서 저를 도와주셨습니다
이사님께서 새로운 정책을 발표하실 예정입니다.
어른들께서는 항상 우리를 걱정하십니다.
선생님께서 질문을 받으십니다.   
할아버지께서 산책을 나가셨어요.
아버지께서 자동차를 운전하십니다."""
final_output = main(text)

In [ ]:
# 18번 예시 
text = """네가 맞아.
나는 지금 밥 먹어.
그는 항상 공부를 열심히 해

네가 맞아.
나는 지금 밥 먹어.
그는 항상 공부를 열심히 해.
내일 꼭 와.
오늘 정말 피곤해.
그 이야기는 나도 들어봤어.
저녁 준비 다 했어.
지금 뭐 하고 있어?
이 문제는 어려워.
우리는 곧 떠나.
"""
final_output = main(text)

In [ ]:
# 19번 예시 
text = """약속을 했으니까 가기 싫어도 갈 수밖에.
이 옷이 작으니까 좀 큰 것으로 바꿔 주세요.

비가 오니까 우산을 챙겨야 해.
시간이 없으니까 빨리 가자.
길이 막히니까 지각할 것 같아.
너무 배가 고프니까 밥을 먹어야겠어.
날씨가 추우니까 따뜻한 옷을 입어.
네가 도와주니까 일이 더 쉬웠어.
약속을 했으니까 반드시 지켜야 해.
오늘은 피곤하니까 일찍 자자.
이 옷이 예쁘니까 살까 고민 중이야.
시간이 늦었으니까 빨리 가자.
"""
final_output = main(text)

In [ ]:
# 20번 예시 
text = """사람들이 많다.

사람들이 모여들었다.
학생들이 운동장에서 뛰고 있다.
책들이 책상 위에 놓여 있다.
여러 가지 문제들이 생겼다.
동물들이 숲에서 살아간다.
친구들이 내 생일을 축하해 줬다.
바람에 나뭇잎들이 흔들리고 있다.
고양이들이 마당에서 놀고 있다.
아이들이 공원에서 놀고 있다.
꽃들이 활짝 피었다.
"""
final_output = main(text)

In [ ]:
# 21번 예시 
text = """오늘이나 내일 중에 편할 때를 선택하여 찾아오거라.
사과나 배 중에서 골라.
버스나 지하철 중에서 빠른 걸로 타고 와.
커피나 차 중에 마시고 싶은 걸 골라.
점심이나 저녁에 만나자.
오늘이나 내일 중에 약속을 잡자.

오늘이나 내일 중에 결정하자.
커피나 차 중에 고를 수 있어.
책이나 영화 중에 선택해.
저녁이나 아침에 만나자.
산책이나 운동을 해 보자.
버스나 지하철을 타고 가자.
사과나 배 중에 어떤 걸 좋아해?
축구나 농구를 할 수 있어.
공책이나 연필을 준비해.
영화나 드라마를 보자.
"""
final_output = main(text)

In [ ]:
# 22번 예시 
text = """백 명이나 모였다고?
사과가 열 개나 있네!
지금까지 이 일을 하려면 열 시간이나 걸린다.
지금까지 이 일을 하려면 한 시간이나 걸린다.
사과를 열 개나 먹었어.
그 책을 세 번이나 읽었어.
오늘만 다섯 시간이나 공부했어.
산 꼭대기까지 한 시간이나 걸렸어.
회의에 사람들이 백 명이나 참석했어.
그 영화가 벌써 천만 명이나 봤대.
가방 무게가 10킬로그램이나 돼.
그곳까지 가는데 두 시간이나 걸려.
가격이 오만 원이나 해!
오늘 기온이 35도나 올라갔어.
그가 벌써 다섯 잔이나 마셨어.
숙제가 열 페이지나 돼.
집에서 학교까지 세 정거장이나 있어.
그 문제를 해결하는 데 한 달이나 걸렸어.
그 식당에서 줄을 서는데 한 시간이나 기다렸어.
이번 달에만 벌써 세 번이나 만났어.
그 선수가 경기에서 열 골이나 넣었어.
그 산의 높이가 3,000미터나 돼.
그 프로젝트에 백만 원이나 투자했어.
오늘 업무가 평소보다 두 배나 많아.
"""
final_output = main(text)

In [ ]:
# 23번 예시 
text = """심심한데 영화나 보러 가자.
시간이 남으니까 책이나 읽어.
밥 없으면 빵이나 먹자.
시간도 남는데 커피나 한잔할까?
다른 특별한 계획이 없어 최소한의 선택을 제안합니다.
심심한데 산책이나 갈래?
특별한 할 일이 없어 산책을 제안합니다.
배도 고픈데 라면이나 먹자.
만족스럽진 않지만 간단한 식사를 제안합니다.
집에 가기 전에 영화나 볼까?
다른 계획이 없으니 영화를 보는 것을 제안합니다.
할 일도 없는데 책이나 읽어야겠다.
특별한 일이 없어서 책을 읽는 것을 선택합니다.
주말에 여행은 못 가니 가까운 곳에 드라이브나 다녀오자.
먼 여행은 어렵지만 가까운 곳을 제안합니다.
옷이 마음에 드는 게 없으니 그냥 이거나 살까?
원하는 것이 없어 차선책을 선택합니다.
시간이 없으니 택시나 타자.
더 좋은 방법이 없어서 택시를 선택합니다.
친구도 없는데 강아지나 키워볼까?
외로움을 달래기 위한 선택을 제안합니다.
비도 오는데 영화나 볼까?
야외 활동이 어려워 다른 선택을 제안합니다.
운동이나 시작해 볼까 생각 중이야.
특별한 계획이 없어 운동을 선택합니다.
할인 기간이 지났으니 정가로나 사야겠어.
할인을 놓쳐 어쩔 수 없이 정가로 사는 상황입니다.
전화 연결이 안 되니 문자나 보내자.
통화가 어려워 다른 방법을 선택합니다.
맛집은 사람이 많으니 근처 식당에서나 먹자.
원하는 곳은 어려워 다른 곳을 선택합니다.
늦었으니 택배는 내일이나 받아야겠다.
오늘은 어려워 내일로 미루는 선택입니다.
모임에 못 가니 선물이나 보내야겠어.
참석이 어려워 다른 방법을 선택합니다.
주차가 힘드니 대중교통이나 이용하자.
불편함을 피하기 위해 다른 방법을 선택합니다.
밖에 덥으니 집에서나 쉬자.
더위를 피하기 위해 집에 있는 것을 선택합니다.
인터넷이 안 되니 책이나 볼 수밖에 없네.
인터넷 사용이 어려워 다른 것을 선택합니다.
약속이 취소되어서 영화나 볼까 했어.
계획이 변경되어 다른 것을 선택합니다.
"""
final_output = main(text)

In [ ]:
# 24번 예시 
text = """공부밖에 모르는 학생"""
# pos_tag_print 함수 호출 # 형태소 분석 수행
final_output = main(text)

In [ ]:
# 25번 예시 
text = """비가 안 온다.

그는 학교에 안 갔다.
오늘 아침에는 안 추웠어.
그 문제는 안 어려워.
나는 그 책을 안 읽었다.
내일은 안 올 거야.
음식이 안 식었다.
그는 지금 집에 안 있어.
그 영화는 별로 안 재미있어.
우리는 이번 주말에 안 만날 거야.
나는 아직 안 결정했어.
"""
final_output = main(text)

In [ ]:
# 26번 예시 
text = """건물 안.

건물 안에 사람이 많다.
책상 안에 서류가 있다.
가방 안에 물건을 넣었다.
방 안이 너무 더워.
박스 안에 뭐가 들어 있을까?
나는 집 안에서 일한다.
차 안에서 대기하고 있어.
상자 안을 확인해 봐.
냉장고 안에 음식이 많아.
지갑 안에 돈이 없었다.
"""
final_output = main(text)

In [ ]:
# 27번 예시 
text = """날이 추운데 따뜻하게 입어.

옷을 따뜻하게 입어.
문제를 쉽게 해결했어.
숙제를 빨리 끝내게 도와줘.
널리 알려지게 할 방법이 뭐야?
이걸 더 재미있게 만들어 봐.
집을 깨끗하게 청소했어.
네가 안전하게 도착하길 바래.
피곤하니까 일찍 자게 해.
시험을 잘 보게 열심히 공부했어.
그렇게 말하지 않게 조심해.
"""
final_output = main(text)

In [ ]:
kiwi = Kiwi(num_workers=0, model_path=None, load_default_dict=True, integrate_allomorph=True, model_type='sbg', typos=None, typo_cost_threshold=2.5)

"""
num_workers: 2 이상이면 단어 추출 및 형태소 분석에 멀티 코어를 활용하여 조금 더 빠른 속도로 분석을 진행할 수 있습니다.
1인 경우 단일 코어만 활용합니다. num_workers가 0이면 현재 환경에서 사용가능한 모든 코어를 활용합니다.
생략 시 기본값은 0입니다.
model_path: 형태소 분석 모델이 있는 경로를 지정합니다. 생략시 kiwipiepy_model 패키지로부터 모델 경로를 불러옵니다.
load_default_dict: 추가 사전을 로드합니다. 추가 사전은 위키백과의 표제어 타이틀로 구성되어 있습니다. 이 경우 로딩 및 분석 시간이 약간 증가하지만 다양한 고유명사를 좀 더 잘 잡아낼 수 있습니다. 분석 결과에 원치 않는 고유명사가 잡히는 것을 방지하려면 이를 False로 설정하십시오.
integrate_allomorph: 어미 중, '아/어', '았/었'과 같이 동일하지만 음운 환경에 따라 형태가 달라지는 이형태들을 자동으로 통합합니다.
model_type: 형태소 분석에 사용할 언어 모델을 지정합니다. 'knlm', 'sbg' 중 하나를 선택할 수 있습니다. 'sbg' 는 상대적으로 느리지만 먼 거리에 있는 형태소 간의 관계를 포착할 수 있습니다.
typos: 형태소 분석 시 간단한 오타를 교정합니다. None으로 설정 시 교정을 수행하지 않습니다.
typo_cost_threshold: 오타 교정을 허용할 최대 오타 비용을 설정합니다.
"""